In [102]:
# pip install faker

In [ ]:
import pandas as pd
import numpy as np
from dataclasses import dataclass
from datetime import datetime, timedelta

import uuid
import math
from itertools import combinations
import random
from faker import Faker
import copy
import ast
from urllib.parse import quote_plus

import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib

import warnings

In [33]:
warnings.filterwarnings(action="ignore")
pd.set_option("display.max_rows", 5000)

# Users

In [ ]:
# Seed 고정
np.random.seed(42)
random.seed(42)
faker = Faker('ko_KR')
Faker.seed(42)

namespace = uuid.NAMESPACE_DNS

# 사용자 수
num_users = random.randint(100_000, 120_000)

# ------------------------
# 1) user_id
# ------------------------
user_id = np.arange(1, num_users + 1)

# ------------------------
# 2) gender (남/여 65:35)
# ------------------------
gender = np.random.choice(['M', 'F'], size=num_users, p=[0.65, 0.35])

# ------------------------
# 3) age (평균 25세, 15-45세 제한)
# ------------------------
main_part = int(num_users * 0.8)
tail_part = num_users - main_part

age_main = np.random.normal(loc=25, scale=3, size=main_part)  # 주요분포: 평균 25, 표준편차 3
age_tail = np.random.normal(loc=30, scale=5, size=tail_part)  # 꼬리분포: 평균 30, 표준편차 5

ages = np.concatenate([age_main, age_tail]) # 결합
np.random.shuffle(ages)

ages = np.clip(ages, 15, 45).round()  # 최종 범위: 15~45로 clip 후 정수화

# ------------------------
# 4) signup_date (20.01.01 - 24.12.31)
# ------------------------
# 연도별 확률 구간 정의
signup_years = [2020, 2021, 2022, 2023, 2024]
year_prob_ranges = {
    2020: (0.01, 0.04),
    2021: (0.03, 0.06),
    2022: (0.15, 0.25),
    2023: (0.35, 0.45),
}

# 구간 내에서 랜덤으로 확률 선택
year_probs = [np.random.uniform(low, high) for (low, high) in year_prob_ranges.values()]
year_probs.append(1 - np.sum(year_probs))

# 가입 연도 생성
signup_year = np.random.choice(signup_years, size=num_users, p=year_probs)

# 월별 확률 설정 (합계 1로 정규화)
month_probs = np.array([0.04, 0.04, 0.06, 0.07, 0.08, 0.12,
                        0.15, 0.15, 0.11, 0.08, 0.06, 0.04])

# 가입 월 생성 (1~12월 중 분포 기반 무작위 선택)
signup_month = np.random.choice(range(1, 13), size=num_users, p=month_probs)

# 가입 연도는 이전 로직에서 생성된 signup_year 활용
signup_date = []
for y, m in zip(signup_year, signup_month):
    start_date = datetime(y, m, 1, 0, 0, 0)
    if m == 12:
        end_date = datetime(y, m, 31, 23, 59, 59)
    else:
        end_date = datetime(y, m + 1, 1, 0, 0, 0) - timedelta(seconds=1)
    dt = faker.date_time_between(start_date=start_date, end_date=end_date)
    signup_date.append(dt.strftime("%Y-%m-%d %H:%M:%S"))

# ------------------------
# 5) birth (2025년 1월 1일 기준 만 나이, age로 산출)
# ------------------------
birth = []
current_date = datetime(2025, 1, 1)
birth_offsets = [timedelta(days=int(age*365 + np.random.randint(0, 364))) for age in ages]
birth = [(current_date - offset).strftime('%Y-%m-%d') for offset in birth_offsets]

# ------------------------
# 5) 주소 (행정구역별 성별 인구통계 기반 랜덤 생성)
# ------------------------
region_df = pd.read_csv('./data/행정구역별_성별_인구통계_2025.csv')

region_pop = region_df.groupby(['시도', '지역구'])['인구수'].sum().reset_index()
region_pop['address'] = region_pop['시도'] + ' ' + region_pop['지역구']

# 확률 계산
region_pop['prob'] = region_pop['인구수'] / region_pop['인구수'].sum()
regions = region_pop['address'].tolist()
probs = region_pop['prob'].tolist()
address = np.random.choice(regions, size=num_users, p=probs)

# ------------------------
# 6) 이름, 이메일 주소, 유입 경로
# ------------------------
first_names = [faker.first_name() for _ in range(num_users)]
last_names = [faker.last_name() for _ in range(num_users)]
emails = [f"{str(uuid.uuid5(namespace, str(uid)))[:10]}@example.com" for uid in user_id]

# traffic_source = np.random.choice(traffic_sources, size=num_users)

# 유입경로 종류
traffic_sources = [
    "direct", "organic_search", "paid_search",
    "facebook", "instagram", "kakao",
    "naver", "email", "referral"
]

# 연도별 유입경로 확률 분포 (2020년 ~ 2024년)
yearly_source_probs = {
    2020: [0.35, 0.40, 0.05, 0.03, 0.04, 0.02, 0.06, 0.01, 0.04],
    2021: [0.40, 0.35, 0.05, 0.02, 0.05, 0.03, 0.05, 0.01, 0.04],
    2022: [0.43, 0.32, 0.04, 0.02, 0.06, 0.03, 0.04, 0.01, 0.05],
    2023: [0.45, 0.30, 0.03, 0.02, 0.07, 0.03, 0.04, 0.01, 0.05],
    2024: [0.46, 0.28, 0.02, 0.02, 0.07, 0.03, 0.04, 0.01, 0.07]
}

# 가입 연도별 확률에 따라 traffic_source 할당
traffic_source = [
    np.random.choice(traffic_sources, p=yearly_source_probs[int(year)])
    for year in signup_year
]

# ------------------------
# 7) DataFrame 생성
# ------------------------
users_df = pd.DataFrame({
    "id": user_id,
    "created_at": signup_date,
    "first_name": first_names,
    "last_name": last_names,
    "email": emails,
    "age": ages,
    "birth": birth,
    "gender": gender,
    "address": address,
    "traffic_source": traffic_source
})

# 속성 타입 변환
users_df["age"] = users_df["age"].astype(int)
users_df["birth"] = pd.to_datetime(users_df["birth"])
users_df["created_at"] = pd.to_datetime(users_df["created_at"])

In [135]:
# CSV로 저장
users_df.to_csv('./data/users.csv', index=False)

# Products

In [156]:
# 서브카테고리별 비중 계산
def build_subcats_weights(category_dict, subcat_weights):
    """
    category_dict : dict
        - {"대분류": ["중분류1", "중분류2", ...]}
    subcat_weights : dict
        - {"대분류": {"중분류명": 비율, ...}} : 지정한 중분류는 비율 반영, 나머지는 균등 분배

    return : dict
        - {"대분류": [중분류별 확률 배열]}
    """
    full_weights = {}

    for category, subcats in category_dict.items():
        if category in subcat_weights:
            weights_dict = subcat_weights[category]
            weights = []
            total_assigned = sum(weights_dict.values())
            remaining_weight = 1.0 - total_assigned

            other_subcats = [s for s in subcats if s not in weights_dict]
            other_weight = remaining_weight / len(other_subcats) if other_subcats else 0

            for s in subcats:
                if s in weights_dict:
                    weights.append(weights_dict[s])
                else:
                    weights.append(other_weight)
        else:
            # 모든 서브카테고리에 균등 분포
            weights = [1 / len(subcats)] * len(subcats)

        full_weights[category] = weights

    return full_weights

In [157]:
# 성별 분류 함수
def assign_department(category, sub_category):
    if category in ["원피스", "스커트"]:
        return "여성"
    if category == "속옷":
        if sub_category in ["브라", "세트 속옷", "속바지"]:
            return "여성"
        elif sub_category in ["팬티", "이너웨어"]:
            p = np.array([0.2, 0.8])
            return np.random.choice(["남성", "여성"], p=p)
        else:
            return "여성"
    if category in ["상의", "아우터", "바지", "신발", "가방", "액세서리"]:
        p = np.array([0.55, 0.35, 0.1])
        return np.random.choice(["남성", "여성", "공용"], p=p)
    return "공용"

In [ ]:
fake = Faker("ko_KR")
np.random.seed(42)

# 카테고리 및 서브카테고리
cat_subcat_dict = {
    "상의": ["반소매 티셔츠", "긴소매 티셔츠", "맨투맨", "셔츠/블라우스", "니트", "후드 티셔츠"],
    "아우터": ["블루종", "레더 재킷", "코트", "패딩", "카디건", "기타 아우터"],
    "바지": ["데님 팬츠", "슬랙스", "조거 팬츠", "숏 팬츠", "기타 바지"],
    "원피스": ["미니 원피스", "셔츠 원피스", "미디 원피스", "니트 원피스", "맥시 원피스"],
    "스커트": ["미니 스커트", "롱 스커트", "미디 스커트"],
    "속옷": ["브라", "팬티", "이너웨어", "세트 속옷", "속바지"],
    "신발": ["스니커즈", "부츠", "로퍼", "운동화", "샌들", "슬리퍼", "구두", "기타 신발"],
    "가방": ["백팩", "크로스백", "숄더백", "기타 가방"],
    "액세서리": ["모자", "벨트", "시계", "양말", "장갑", "안경/선글라스", "기타 악세사리"]
}

# 서브카테고리 가중치 지정
subcat_weights = {
    "상의": {"반소매 티셔츠": 0.3, "긴소매 티셔츠": 0.25},
    "바지": {"데님 팬츠": 0.35, "슬랙스": 0.3},
    "신발": {"스니커즈": 0.3, "운동화": 0.25},
    "가방": {"백팩": 0.4},
    "액세서리": {"모자": 0.4}
}

# 가격 범위
price_ranges = {
    "상의": {"반소매 티셔츠": (15000, 39000), "긴소매 티셔츠": (20000, 49000), "맨투맨": (30000, 69000),
           "셔츠/블라우스": (30000, 79000), "니트": (35000, 99000), "후드 티셔츠": (35000, 89000)},
    "아우터": {"블루종": (50000, 129000), "레더 재킷": (99000, 199000), "코트": (99000, 250000),
             "패딩": (109000, 300000), "카디건": (39000, 99000), "기타 아우터": (49000, 159000)},
    "바지": {"데님 팬츠": (39000, 99000), "슬랙스": (39000, 89000), "조거 팬츠": (30000, 79000),
           "숏 팬츠": (25000, 59000), "기타 바지": (29000, 79000)},
    "원피스": {"미니 원피스": (40000, 99000), "셔츠 원피스": (49000, 109000), "미디 원피스": (49000, 119000),
             "니트 원피스": (59000, 139000), "맥시 원피스": (59000, 149000)},
    "스커트": {"미니 스커트": (30000, 69000), "롱 스커트": (39000, 89000), "미디 스커트": (39000, 79000)},
    "속옷": {"브라": (15000, 59000), "팬티": (5000, 19000), "이너웨어": (10000, 35000),
           "세트 속옷": (25000, 69000), "속바지": (10000, 29000)},
    "신발": {"스니커즈": (45000, 139000), "부츠": (69000, 199000), "로퍼": (59000, 139000),
           "운동화": (59000, 189000), "샌들": (29000, 79000), "슬리퍼": (19000, 49000),
           "구두": (69000, 159000), "기타 신발": (30000, 129000)},
    "가방": {"백팩": (45000, 129000), "크로스백": (35000, 99000), "숄더백": (39000, 109000),
           "기타 가방": (30000, 99000)},
    "액세서리": {"벨트": (19000, 59000), "시계": (69000, 259000),
              "양말": (2000, 10000), "장갑": (5000, 29000), "안경/선글라스": (29000, 129000),
              "기타 악세사리": (5000, 49000), "모자": (15000, 49000)}
}

# 카테고리별 브랜드
category_brands = {
  "상의": ["Musinsa Standard", "Ouro", "Curetty", "RonRon", "J Vineyard", "Seez", "Kijiko", "Lowbi", "MuahMuah", "Lohnt",
          "Holiday Outerwear", "ATAR", "Loiter Loiter", "Remain", "Pakke", "Arts de Base", "Thehere", "Moderate", "Nike",
          "Adidas", "Stussy", "The North Face", "A Bathing Ape (BAPE)", "Supreme", "Carhartt WIP", "Vivienne Westwood",
          "Arc’teryx", "New Balance", "Asics", "Oakley", "Palace", "MISCHIEF", "Human Made", "Converse", "Vans", "Puma",
          "Fila", "Reebok", "Under Armour", "Discovery Expedition", "National Geographic Apparel", "Black Yak", "K2",
          "Patagonia", "Columbia", "Stone Island", "Off-White", "thisisneverthat", "Andersson Bell", "87MM", "Covernat",
          "LMC", "Mahagrid", "Kirsh", "5252 by O!Oi", "ADLV", "Nerdy", "MLB", "New Era", "SPAO", "Lucky Chouette", "CC Collect",
          "Guess", "Levi’s", "Calvin Klein", "Tommy Hilfiger", "Ralph Lauren", "Lacoste", "Beyond Closet", "SJYP", "pushBUTTON",
          "Minjukim", "YCH", "We11Done", "ADER Error", "Maje", "Iro", "Kuho", "ORR", "Egoist", "EENK", "Jill Stuart",
          "Margarin Fingers", "Sandro", "DEW E DEW E", "Salomon", "Moncler", "Beanpole", "Kolon Sport", "Hazzys", "Daks",
          "Henry Cotton’s", "Club Monaco", "Brooks Brothers", "Teenie Weenie", "Series", "Polo Jeans (Ralph Lauren)",
          "Tommy Jeans", "McGregor", "Suitsupply", "Champion", "XEXYMIX"],
  "아우터": ["Musinsa Standard", "Ouro", "Curetty", "RonRon", "J Vineyard", "Seez", "Kijiko", "Lowbi", "MuahMuah", "Lohnt",
           "Holiday Outerwear", "ATAR", "Loiter Loiter", "Remain", "Pakke", "Arts de Base", "Thehere", "Moderate", "Nike",
           "Adidas", "Stussy", "The North Face", "A Bathing Ape (BAPE)", "Supreme", "Carhartt WIP", "Vivienne Westwood",
           "Arc’teryx", "New Balance", "Asics", "Oakley", "Palace", "MISCHIEF", "Human Made", "Converse", "Vans", "Puma",
           "Fila", "Reebok", "Under Armour", "Discovery Expedition", "National Geographic Apparel", "Black Yak", "K2",
           "Patagonia", "Columbia", "Stone Island", "Off-White", "thisisneverthat", "Andersson Bell", "87MM", "Covernat",
           "LMC", "Mahagrid", "Kirsh", "5252 by O!Oi", "ADLV", "Nerdy", "MLB", "New Era", "SPAO", "Lucky Chouette", "CC Collect",
           "Guess", "Levi’s", "Calvin Klein", "Tommy Hilfiger", "Ralph Lauren", "Lacoste", "Beyond Closet", "SJYP", "pushBUTTON",
           "Minjukim", "YCH", "We11Done", "ADER Error", "Maje", "Iro", "Kuho", "ORR", "Egoist", "EENK", "Jill Stuart",
           "Margarin Fingers", "Sandro", "DEW E DEW E", "Salomon", "Moncler", "Beanpole", "Kolon Sport", "Hazzys", "Daks",
           "Henry Cotton’s", "Club Monaco", "Brooks Brothers", "Teenie Weenie", "Series", "Polo Jeans (Ralph Lauren)",
           "Tommy Jeans", "McGregor", "Suitsupply", "Champion", "XEXYMIX"],
  "바지": ["Musinsa Standard", "Ouro", "Curetty", "RonRon", "J Vineyard", "Seez", "Kijiko", "Lowbi", "MuahMuah", "Holiday Outerwear",
          "Loiter Loiter", "Remain", "Pakke", "Arts de Base", "Thehere", "Moderate", "Nike", "Adidas", "Stussy", "The North Face",
          "A Bathing Ape (BAPE)", "Supreme", "Carhartt WIP", "Vivienne Westwood", "Arc’teryx", "New Balance", "Asics", "Oakley",
          "Palace", "MISCHIEF", "Human Made", "Converse", "Vans", "Puma", "Fila", "Reebok", "Under Armour", "Discovery Expedition",
          "National Geographic Apparel", "Black Yak", "K2", "Patagonia", "Columbia", "Stone Island", "Off-White", "thisisneverthat",
          "Andersson Bell", "87MM", "Covernat", "LMC", "Mahagrid", "Kirsh", "5252 by O!Oi", "ADLV", "Nerdy", "MLB", "SPAO",
          "Lucky Chouette", "CC Collect", "Guess", "Levi’s", "Calvin Klein", "Tommy Hilfiger", "Ralph Lauren", "Lacoste",
          "Beyond Closet", "SJYP", "pushBUTTON", "Minjukim", "YCH", "We11Done", "ADER Error", "Maje", "Iro", "Kuho", "ORR", "Egoist",
          "EENK", "Jill Stuart", "Margarin Fingers", "Sandro", "DEW E DEW E", "Salomon", "Moncler", "Beanpole", "Kolon Sport", "Hazzys",
          "Daks", "Henry Cotton’s", "Club Monaco", "Brooks Brothers", "Teenie Weenie", "Series", "Polo Jeans (Ralph Lauren)",
          "Tommy Jeans", "McGregor", "Suitsupply", "Champion", "XEXYMIX"],
  "가방": ["Musinsa Standard", "Ouro", "Curetty", "RonRon", "J Vineyard", "Lohnt", "Arts de Base", "Thehere", "Nike", "Adidas", "Stussy",
          "The North Face", "A Bathing Ape (BAPE)", "Supreme", "Carhartt WIP", "Vivienne Westwood", "Arc’teryx", "New Balance", "Asics",
          "Oakley","Converse", "Dr. Martens", "Puma", "Fila", "Reebok", "Discovery Expedition", "National Geographic Apparel", "Black Yak",
          "K2", "Patagonia","Columbia", "Off-White", "Kirsh", "MLB", "Lucky Chouette", "Guess", "Calvin Klein", "Tommy Hilfiger",
          "Ralph Lauren", "Lacoste", "Marhen J", "OSOI", "Maje", "ORR", "EENK", "Jill Stuart", "Sandro", "Salomon", "Beanpole", "Kolon Sport",
          "Hazzys", "Daks", "Henry Cotton’s"],
  "액세서리": ["Musinsa Standard", "Curetty", "RonRon", "J Vineyard", "Seez", "MuahMuah", "Holiday Outerwear", "ATAR", "Loiter Loiter", "Pakke",
             "Arts de Base", "Thehere", "Nike", "Adidas", "Stussy", "The North Face", "A Bathing Ape (BAPE)", "Supreme", "Carhartt WIP",
             "Vivienne Westwood", "Arc’teryx", "New Balance", "Asics", "Oakley", "Palace", "MISCHIEF", "Human Made", "Converse", "Vans",
             "Dr. Martens", "Puma", "Fila", "Reebok", "Under Armour", "Discovery Expedition", "National Geographic Apparel", "Black Yak",
             "K2", "Patagonia", "Columbia", "Stone Island", "Off-White", "thisisneverthat", "Andersson Bell", "87MM", "Covernat", "LMC",
             "Mahagrid", "Kirsh", "5252 by O!Oi", "ADLV", "Nerdy", "MLB", "New Era", "SPAO", "Lucky Chouette", "Guess", "Levi’s", "Calvin Klein",
             "Tommy Hilfiger", "Ralph Lauren", "Lacoste", "Beyond Closet", "SJYP", "pushBUTTON", "We11Done", "ADER Error", "Marhen J", "OSOI",
             "Maje", "Iro", "EENK", "Jill Stuart", "Margarin Fingers", "Sandro", "DEW E DEW E", "Salomon", "Moncler", "Beanpole", "Kolon Sport",
             "Hazzys", "Daks", "Henry Cotton’s", "Club Monaco", "Brooks Brothers", "Teenie Weenie", "Series", "Polo Jeans (Ralph Lauren)",
             "Tommy Jeans", "McGregor", "Suitsupply", "Champion", "XEXYMIX"],
  "속옷": ["Musinsa Standard", "SPAO", "Calvin Klein", "XEXYMIX"],
  "신발": ["Musinsa Standard", "RonRon", "Lohnt", "Nike", "Adidas", "Stussy", "The North Face", "A Bathing Ape (BAPE)", "Supreme", "Carhartt WIP",
          "Vivienne Westwood", "Arc’teryx", "New Balance", "Asics", "Oakley", "Palace", "Human Made", "Converse", "Vans", "Dr. Martens", "Puma",
          "Fila", "Reebok", "Under Armour", "Discovery Expedition", "National Geographic Apparel", "Black Yak", "K2", "Patagonia", "Columbia",
          "Off-White", "thisisneverthat", "Kirsh", "Nerdy", "MLB", "Lucky Chouette", "Guess", "Levi’s", "Calvin Klein", "Tommy Hilfiger",
          "Ralph Lauren", "Lacoste", "We11Done", "ADER Error", "OSOI", "Salomon", "Moncler", "Beanpole", "Kolon Sport", "Hazzys", "Daks",
          "Henry Cotton’s", "Brooks Brothers", "Series", "Polo Jeans (Ralph Lauren)", "Tommy Jeans", "McGregor", "Champion"],
  "원피스": ["Ouro", "Curetty", "RonRon", "J Vineyard", "Kijiko", "MuahMuah", "Remain", "Pakke", "Arts de Base", "Thehere", "Moderate", "Nike",
           "Adidas", "Stussy", "MISCHIEF", "Converse", "Andersson Bell", "Kirsh", "5252 by O!Oi", "Lucky Chouette", "CC Collect", "SJYP", "pushBUTTON",
           "Minjukim", "YCH", "Maje", "Iro", "Kuho", "ORR", "Egoist", "EENK", "Jill Stuart", "Margarin Fingers", "Sandro", "DEW E DEW E", "Beanpole",
           "Club Monaco", "Teenie Weenie"],
  "스커트": ["Ouro", "Curetty", "RonRon", "J Vineyard", "Kijiko", "MuahMuah", "Remain", "Pakke", "Arts de Base", "Thehere", "Moderate", "Nike",
           "Adidas", "Stussy", "MISCHIEF", "Converse", "Andersson Bell", "Kirsh", "5252 by O!Oi", "Lucky Chouette", "CC Collect", "SJYP",
           "pushBUTTON", "Minjukim", "YCH", "Maje", "Iro", "Kuho", "ORR", "Egoist", "EENK", "Jill Stuart", "Margarin Fingers", "Sandro",
           "DEW E DEW E", "Beanpole", "Club Monaco", "Teenie Weenie"]
}


# 원가 비율 
cost_rate_ranges = {
    "상의": (0.45, 0.55),
    "아우터": (0.40, 0.50),
    "바지": (0.45, 0.55),
    "원피스": (0.40, 0.50),
    "스커트": (0.45, 0.55),
    "속옷": (0.50, 0.60),
    "신발": (0.40, 0.50),
    "가방": (0.40, 0.50),
    "액세서리": (0.50, 0.60)
}

# 제품 수 및 비율
category_ratio = {
    "상의": 0.22, "아우터": 0.15, "바지": 0.12, "원피스": 0.08,
    "스커트": 0.05, "속옷": 0.06, "신발": 0.13, "가방": 0.10, "액세서리": 0.09
}

# 제품 생성
total_products = 30000

full_weights = build_subcats_weights(cat_subcat_dict, subcat_weights)
products = []
product_names = []
namespace = uuid.NAMESPACE_DNS

for category, subcats in cat_subcat_dict.items():
    cat_count = int(total_products * category_ratio[category])
    weights = full_weights[category]
    subcat_counts = np.random.multinomial(cat_count, weights)
    brand_pool = category_brands[category]
    cost_low, cost_high = cost_rate_ranges[category]
    
    for subcat, count in zip(subcats, subcat_counts):
        for _ in range(count):
            brand = np.random.choice(brand_pool)
            department = assign_department(category, subcat)
            color = fake.color_name()
            num = np.random.randint(1, 1000)
            name = f"{brand} {subcat} {color} {num}"
            
            # name 중복 예외 처리
            if name in product_names:
                num = np.random.randint(1001, 2000)
                name = f"{brand} {subcat} {color} {num}"

            product_names.append(name)

            low, high = price_ranges[category][subcat]
            retail_price = (np.random.randint(low, high + 1) // 100) * 100
            cost_factor = np.random.uniform(cost_low, cost_high)
            cost = int((retail_price * cost_factor) // 100) * 100  # 원가도 100원 단위로 반올림

            product_id = str(uuid.uuid5(namespace, str(name)))[:8]

            products.append({
                "id": product_id,
                "category": category,
                "sub_category": subcat,
                "name": name,
                "brand": brand,
                "cost": cost,
                "retail_price": retail_price,
                "department": department
            })

# DataFrame 생성
products_df = pd.DataFrame(products)

In [171]:
products_df.to_csv("./data/products.csv", index=False)

# Promotions

In [ ]:
# 할인율 상수
ALWAYS_ON_DISCOUNT = 0.10
SEASONAL_DISCOUNT = 0.20

# 최소 주문 금액 
ALWAYS_MINIMUM_SALE_PRICE = 0
SEASONAL_MINIMUM_SALE_PRICE = 50000

# 최대 할인 금액 
ALWAYS_MAXIMUM_DISCOUNT_PRICE = 300000
SEASONAL_MAXIMUM_DISCOUNT_PRICE = 300000

# 프로모션 타입
ALWAYS_PROMOTION = '상시'
SEASONAL_PROMOTION = '정기'

# 프로모션 리스트 생성 
promotions_lst = []

# 타임 세일 프로모션 생성 
time_sale = ("타임세일", datetime(2022, 1, 1), datetime(2024, 12, 31), ALWAYS_ON_DISCOUNT, ALWAYS_MINIMUM_SALE_PRICE, ALWAYS_MAXIMUM_DISCOUNT_PRICE, ALWAYS_PROMOTION)
promotions_lst.append(time_sale)


# 22, 23, 24년 설날 날짜
new_years_dates = {
    2022: "2022-02-01",
    2023: "2023-01-22",
    2024: "2024-02-10"
}

# 설날 프로모션 생성 
for year, date in new_years_dates.items():
    date_datetime = datetime.strptime(date, "%Y-%m-%d")
    start, end = date_datetime + timedelta(days=2), date_datetime + timedelta(days=8)
    promotions_lst.append(("설날 프로모션", start, end, SEASONAL_DISCOUNT, SEASONAL_MINIMUM_SALE_PRICE, SEASONAL_MAXIMUM_DISCOUNT_PRICE, SEASONAL_PROMOTION))

# 22, 23, 24년 추석 날짜 
chuseok_dates = {
    2022: "2022-09-10",
    2023: "2023-09-29",
    2024: "2024-09-17"
}

# 추석 프로모션 생성 
for year, date in chuseok_dates.items():
    date_datetime = datetime.strptime(date, "%Y-%m-%d")
    start, end = date_datetime + timedelta(days=2), date_datetime + timedelta(days=8)
    promotions_lst.append(("추석 프로모션", start, end, SEASONAL_DISCOUNT, SEASONAL_MINIMUM_SALE_PRICE, SEASONAL_MAXIMUM_DISCOUNT_PRICE, SEASONAL_PROMOTION))

# 봄 블랙프라이데이 생성
for year in [2022, 2023, 2024]:
    first_day = datetime(year, 5, 1)
    first_monday = first_day + timedelta(days=(7 - first_day.weekday()) % 7)
    start = first_monday + timedelta(weeks=1)
    end = start + timedelta(days=13)
    promotions_lst.append(("봄 블랙프라이데이", start, end, SEASONAL_DISCOUNT, SEASONAL_MINIMUM_SALE_PRICE, SEASONAL_MAXIMUM_DISCOUNT_PRICE, SEASONAL_PROMOTION))

# 가을 블랙프라이데이 생성
for year in [2022, 2023, 2024]:
    first_day = datetime(year, 11, 1)
    first_monday = first_day + timedelta(days=(7 - first_day.weekday()) % 7)
    start = first_monday + timedelta(weeks=2)
    end = start + timedelta(days=13)
    promotions_lst.append(("가을 블랙프라이데이", start, end, SEASONAL_DISCOUNT, SEASONAL_MINIMUM_SALE_PRICE, SEASONAL_MAXIMUM_DISCOUNT_PRICE, SEASONAL_PROMOTION))


# 생일자 프로모션
birth_sale = ("생일자 프로모션", datetime(2022, 1, 1), datetime(2024, 12, 31), SEASONAL_DISCOUNT, SEASONAL_MINIMUM_SALE_PRICE, SEASONAL_MAXIMUM_DISCOUNT_PRICE, ALWAYS_PROMOTION)
promotions_lst.append(birth_sale)


promotions = []
for i, (name, start, end, discount_rate, minimum_sale_price, maximum_discount_price, promotion_type) in enumerate(promotions_lst, 1):
    promotions.append({
        "id": i,
        "name": name,
        "promotion_type": promotion_type,
        "start_date": start.strftime("%Y-%m-%d"),
        "end_date": end.strftime("%Y-%m-%d"),
        "discount_rate": discount_rate,
        "minimum_sale_price": minimum_sale_price,
        "maximum_discount_price": maximum_discount_price
    })
    
promotions_df = pd.DataFrame(promotions)

# 컬럼 속성 변경
promotions_df['start_date'] = pd.to_datetime(promotions_df['start_date'])
promotions_df['end_date'] = pd.to_datetime(promotions_df['end_date'])

# 할인 없음 데이터 생성
no_promo = {
    "id": -1,
    "name": "할인 없음",
    "promotion_type": "-",
    "start_date": pd.NaT,          # 기간 없음 → NULL
    "end_date": pd.NaT,            # 기간 없음 → NULL
    "discount_rate": 0.0,
    "minimum_sale_price": 0,
    "maximum_discount_price": 0
}

promotions_df = pd.concat([promotions_df, pd.DataFrame([no_promo])], ignore_index=True)
promotions_df = promotions_df.sort_values(by='id', ascending=True).reset_index(drop=True)

In [89]:
promotions.to_csv("./data/promotions.csv", index=False)

# Events

## 함수 정의

In [4]:
def generate_prob_from_range(range_dict):
    """
    주어진 범위(range_dict)에 따라 각 항목의 확률값을 생성하고, 전체 합이 1이 되도록 정규화된 확률 분포를 반환합니다.

    input:
        range_dict (dict): 각 항목에 대한 확률 범위를 지정하는 딕셔너리 (예: {"A": (0.1, 0.3), "B": (0.2, 0.4)})

    return:
        dict: 정규화된 확률값을 가진 딕셔너리 (예: {"A": 0.4, "B": 0.6})
    """
    raw_probs = [np.random.uniform(low, high) for low, high in range_dict.values()]
    norm_probs = np.array(raw_probs) / np.sum(raw_probs)
    return dict(zip(range_dict.keys(), norm_probs))


def adjust_traffic_for_month(base_probs, month):
    """
    특정 월(month)에 따라 트래픽 소스별 비율을 조정한 후, 정규화된 확률분포를 반환합니다.

    input:
        base_probs (dict): 월과 무관한 기본 트래픽 소스별 비율
        month (int): 조정할 월 (1~12)

    return:
        dict: 월별 특성을 반영하여 조정된 트래픽 소스별 확률 분포
    """
    adj_probs = base_probs.copy()

    # 1~2월: 검색 트래픽 강화, 유료 검색 감소
    if month in [1, 2]:
        adj_probs["organic_search"] *= 1.2
        adj_probs["paid_search"] *= 0.8
    # 6~8월: 소셜 및 유료 검색 강화, 검색 감소
    elif month in [6, 7, 8]:
        for k in ["facebook", "instagram", "kakao", "paid_search"]:
            adj_probs[k] *= 1.3
        adj_probs["organic_search"] *= 0.7
    # 11~12월: 직접 유입 및 유료 검색 강화, 검색 감소
    elif month in [11, 12]:
        adj_probs["direct"] *= 1.3
        adj_probs["paid_search"] *= 1.4
        adj_probs["organic_search"] *= 0.6

    # 정규화
    total = sum(adj_probs.values())
    return {k: v / total for k, v in adj_probs.items()}


def generate_address_by_visitor_stats(created_at, visitor_df):
    """
    방문 시간(created_at)에 따라 월별 방문자 통계를 기반으로 한 행정구역별 주소(address)를 무작위로 생성합니다.

    input:
        created_at (datetime): 이벤트 발생 시각
        visitor_df (pd.DataFrame): 연도, 월, 행정구역별 방문자 수 통계가 포함된 데이터프레임
            - 필수 컬럼: 'year', 'month', 'address', 'touNum' (방문자 수)

    return:
        str: 선택된 행정구역명 (예: "서울특별시 강남구")
    """
    evt_year = created_at.year
    evt_month = created_at.month

    # 해당 연/월 데이터 필터링
    filtered = visitor_df[(visitor_df['year'] == evt_year) & (visitor_df['month'] == evt_month)]
    if filtered.empty:
        # 월별 데이터가 없을 경우 전체 방문자 수 기반으로 address 비율 계산
        filtered = visitor_df.groupby("address").agg({"touNum": "sum"}).reset_index()
        total = filtered["touNum"].sum()
        filtered["touRatio"] = filtered["touNum"] / total

    addresses = filtered['address'].tolist()
    probs = filtered['touRatio'].values
    probs = probs / probs.sum()  # 정규화된 확률

    return np.random.choice(addresses, p=probs)

In [5]:
def event_time_gap(prev_event, curr_event):
    """
    이전 이벤트(prev_event)와 현재 이벤트(curr_event) 사이의 시간 간격(초 단위)을 
    사용자 행동 특성에 기반하여 무작위로 생성합니다.

    의류 e-commerce 플랫폼 사용자 흐름을 기반으로 이벤트 쌍별 시간 간격 분포를 반영합니다.
    예를 들어 상품 상세 페이지(view_product) 이후 장바구니(add_to_cart)까지의 시간은 평균 1.5분 이상 소요되며,
    로그인 이후 특정 행동까지는 수 초 ~ 수십 초 내외로 발생합니다.

    input:
        prev_event (str): 직전 이벤트 타입 (예: "view_product")
        curr_event (str): 현재 이벤트 타입 (예: "add_to_cart")

    return:
        float: prev_event와 curr_event 사이의 시간 간격 (단위: 초)
    """
    if (prev_event, curr_event) == ("add_to_cart", "cart"):
        return np.random.uniform(0, 3)
    elif (prev_event, curr_event) == ("cart", "remove_from_cart"):
        return np.random.exponential(scale=60)
    elif (prev_event, curr_event) == ("click_promotion", "login"):
        return np.random.exponential(scale=20)
    elif (prev_event, curr_event) == ("click_promotion", "signup"):
        return np.random.exponential(scale=45)
    elif (prev_event, curr_event) == ("click_promotion", "view_product"):
        return abs(np.random.normal(loc=30, scale=10))
    elif (prev_event, curr_event) == ("login", "add_to_cart"):
        return np.random.uniform(0, 3)
    elif (prev_event, curr_event) == ("login", "cart"):
        return np.random.uniform(0, 3)
    elif (prev_event, curr_event) == ("login", "checkout"):
        return np.random.uniform(0, 3)
    elif (prev_event, curr_event) == ("login", "click_promotion"):
        return np.random.exponential(scale=12)
    elif (prev_event, curr_event) == ("login", "page_view"):
        return abs(np.random.normal(loc=7, scale=3))
    elif (prev_event, curr_event) == ("login", "profile"):
        return np.random.uniform(0, 3)
    elif (prev_event, curr_event) == ("login", "search"):
        return np.random.exponential(scale=8)
    elif (prev_event, curr_event) == ("login", "view_product"):
        return abs(np.random.normal(loc=15, scale=5))
    elif (prev_event, curr_event) == ("page_view", "login"):
        return abs(np.random.normal(loc=4, scale=2))
    elif (prev_event, curr_event) == ("page_view", "signup"):
        return np.random.exponential(scale=40)
    elif (prev_event, curr_event) == ("remove_from_cart", "cart"):
        return np.random.uniform(0, 3)
    elif (prev_event, curr_event) == ("search", "login"):
        return np.random.exponential(scale=20)
    elif (prev_event, curr_event) == ("search", "view_product"):
        return np.random.exponential(scale=45)
    elif (prev_event, curr_event) == ("session_start", "click_promotion"):
        return np.random.uniform(0, 3)
    elif (prev_event, curr_event) == ("session_start", "page_view"):
        return np.random.uniform(0, 3)
    elif (prev_event, curr_event) == ("session_start", "view_product"):
        return np.random.uniform(0, 3)
    elif (prev_event, curr_event) == ("signup", "click_promotion"):
        return np.random.exponential(scale=30)
    elif (prev_event, curr_event) == ("signup", "page_view"):
        return abs(np.random.normal(loc=25, scale=10))
    elif (prev_event, curr_event) == ("signup", "search"):
        return abs(np.random.normal(loc=20, scale=5))
    elif (prev_event, curr_event) == ("signup", "view_product"):
        return abs(np.random.normal(loc=30, scale=10))
    elif (prev_event, curr_event) == ("view_product", "add_to_cart"):
        return np.random.exponential(scale=90)
    elif (prev_event, curr_event) == ("view_product", "checkout"):
        return np.random.exponential(scale=120)
    elif (prev_event, curr_event) == ("view_product", "login"):
        return np.random.exponential(scale=15)
    elif (prev_event, curr_event) == ("view_product", "signup"):
        return np.random.exponential(scale=60)
    elif (prev_event, curr_event) == ("checkout", "purchase"):
        return abs(np.random.normal(loc=120, scale=60))
    elif prev_event == curr_event:
        return np.random.exponential(scale=20)  # 동일 이벤트 반복
    else:
        return np.random.exponential(scale=6)  # scroll 가정

In [6]:
def generate_search_event_count():
    """
    국내 의류 e-commerce 사용자 특성 기반 세션당 검색 이벤트 수를 생성합니다.
    
    - 전체 세션 중 약 35%는 검색을 수행
    - 검색한 세션 중 약 22%는 2회 이상 반복 검색
    
    return: int (search 이벤트 수)
    """
    base_prob = random.random()

    if base_prob < 0.35:
        # 검색 수행 세션
        repeat_prob = random.random()
        if repeat_prob < 0.22:
            return random.choice([2, 3])  # 반복 검색
        else:
            return 1  # 1회 검색
    else:
        return 0  # 검색 없음

In [ ]:
def generate_session_events(session_id, user_id, user_is_new, source, session_time, promotion_df, forced_event_type=None):
    """
    하나의 사용자 세션에서 발생할 고객 행동 이벤트 시퀀스를 생성합니다.

    사용자의 방문 유형(신규/기존), 트래픽 소스(direct, 검색, SNS 등), 탐색 및 구매 여부에 따라 
    search, page_view, view_product, add_to_cart, remove_from_cart, purchase 등의 실제적인 행동 흐름을 구성합니다.
    각 이벤트는 사용자의 특성 및 세션 길이에 기반하여 확률적으로 삽입됩니다.

    input:
        session_id (str): 세션의 고유 ID (사용자 구분 및 연관 관계 추적용)
        user_id (int): 해당 세션의 사용자 ID
        user_is_new (bool): True일 경우 신규 유저로 간주되어 signup 이벤트가 포함됩니다
        source (str): 해당 세션의 트래픽 유입 경로 (예: 'direct', 'organic_search', 'kakao' 등)

    return:
        list[str]: 세션 내 사용자 행동 이벤트 시퀀스 리스트
                   (예: ['page_view', 'view_product', 'add_to_cart', 'purchase', 'logout'])
    """
    events = []

    # 1. 초기 진입 이벤트 결정 (채널별 첫 이벤트 설정)
    if source == 'direct':
        # Direct 유입 시 첫 이벤트는 page_view (예: 홈페이지 방문)
        events.append('page_view')
    else:
        # 검색 또는 SNS 유입 시 첫 이벤트를 랜덤 선택 (page_view, view_product, click_promotion 중 하나)
        first_event_candidates = ['page_view', 'view_product', 'click_promotion']
        events.append(random.choice(first_event_candidates))

    # 2. 이후 이벤트 구성:
    # 세션 길이를 임의로 결정하고 이벤트들을 랜덤하게 선택하되 구매/장바구니 관련 규칙 적용
    # 세션 길이가 짧으면 탐색 위주 이벤트만 포함하도록 조정

    year = session_time.year

    regular_promotions = promotion_df[(promotion_df['promotion_type']=='정기') & (promotion_df['start_date'].dt.year ==year)]
    regular_promotions = regular_promotions[(regular_promotions['start_date'] <= session_time) & (regular_promotions['end_date'] >= session_time)]
    
    user_birth_month = users_df.loc[users_df['id']==user_id, 'birth'].dt.month.values[0]

    if (len(regular_promotions) > 0) or (session_time.month == user_birth_month):
        probs = (
		    [0.03, 0.03, 0.04] +             # 3~5
		    [0.09, 0.1, 0.1, 0.09, 0.08] +   # 6~10
		    [0.07, 0.06, 0.05] +             # 11~13
		    [0.04, 0.03, 0.02] +             # 14~16
		    [0.01, 0.01]                     # 17~18
		)
    else:
        probs = [0.05]*3 + [0.08]*4 + [0.06]*3 + [0.03]*3 + [0.01]*3
        
    probs /= np.sum(probs)

    session_length = np.random.choice(
        list(range(3, 19)),
        p=probs
    )

    # 구매 및 장바구니 관련 플래그 초기화
    has_purchase = False
    has_add_to_cart = False

    # 구매 여부 결정: 세션 이벤트가 6개 이상인 경우에만 구매 시나리오 고려 (짧은 세션은 구매 없음)
    if session_length > 5:
        # 예: 30% 확률로 구매 이벤트를 포함하는 세션으로 지정
        has_purchase = (random.random() < 0.3)
    else:
        has_purchase = False  # 5개 이하 이벤트 세션은 구매 없음

    # 구매 시나리오가 아니면 장바구니 담기만 하는 경우를 소량 허용 (낮은 확률)
    num_add_to_cart_events = 0
    if has_purchase:
        # 구매 세션인 경우: 최소 1개 이상의 add_to_cart 발생
        num_add_to_cart_events = random.randint(1, 3)
        has_add_to_cart = True  # 구매 세션이므로 add_to_cart 이벤트는 반드시 발생
    else:
        # 구매 없는 세션에서 가끔 장바구니 담기만 하는 시나리오 발생 (세션이 충분히 길 때 낮은 확률)
        if session_length > 5 and random.random() < 0.2:
            has_add_to_cart = True
            num_add_to_cart_events = random.randint(1, 2)  # 구매 없이 담기만 하는 경우 1-2회

    # 3. 메인 이벤트 흐름 작성

    # 탐색 중심 이벤트 생성
    for _ in range(session_length):
        events.append(random.choice(['page_view', 'view_product', 'click_promotion']))

    # first_event가 'view_product'인 경우, 10-30% 확률로 직후에 add_to_cart 삽입
    if events[0] == 'view_product' and random.random() < 0.2:
        events.insert(1, 'add_to_cart')
        has_add_to_cart = True

        if num_add_to_cart_events > 0:
            num_add_to_cart_events -= 1

    # 검색 이벤트 추가
    num_search_events = generate_search_event_count()
    if events[1] == 'add_to_cart':
        search_indices = random.choices(range(2, len(events)+1), k=num_search_events)
    else:
        search_indices = random.choices(range(1, len(events)+1), k=num_search_events)
    for index in search_indices:
        events.insert(index, 'search')

    # add_to_cart 삽입
    if has_add_to_cart:
        # 예외처리 : has_add_to_cart인데, 랜덤 생성 과정에서 view_product가 생성되지 않은 경우, view_product 추가
        if 'view_product' not in events:
            vp_valid_indices = [idx for idx, event in enumerate(events) if event in ['page_view', 'search', 'click_promotion']]
            events.insert(random.choice(vp_valid_indices)+1, 'view_product')

        added_cart = 0
        view_product_indices = [index for index, event in enumerate(events) if event == 'view_product']

        # 예외처리 : 이벤트의 시작 sequence가 view_product → add_to_cart 일 경우, view_product_indices에서 0번 인덱스 제거
        if (events[0] == 'view_product') and (events[1] == 'add_to_cart'):
            view_product_indices.remove(0)

        num_add_to_cart_events = min(num_add_to_cart_events, len(view_product_indices))

        while added_cart < num_add_to_cart_events:
            # view_product 바로 뒤에 add_to_cart 삽입
            view_product_index = random.choice(view_product_indices)
            events.insert(view_product_index+1, 'add_to_cart')
            view_product_indices.remove(view_product_index)
            view_product_indices = [idx + 1 if idx > view_product_index else idx for idx in view_product_indices]

            added_cart += 1

        has_add_to_cart = 'add_to_cart' in events

    # 4. purchase 삽입 (has_purchase True일 경우, add_to_cart 이후 랜덤한 위치)
    if has_purchase and has_add_to_cart:
        # add_to_cart 이후 위치에 삽입
        cart_indices = [index for index, event in enumerate(events) if event == 'add_to_cart']
        invalid_indices = []
        for idx in cart_indices:
            if len(invalid_indices) == 0:
                invalid_indices.extend(list(range(0, idx+1)))  # purchase는 최초 add_to_cart보다 선행할 수 없음
            else:
                invalid_indices.extend(list(range(idx-1, idx+1)))  # page_view → view_product → add_to_cart 사이에 위치할 수 없음
        
        # purchase 이벤트가 위치할 수 없는 인덱스 중복 제거 및 정렬
        invalid_indices = set(invalid_indices)
        invalid_indices = sorted(list(invalid_indices))

        pc_valid_indices = [idx for idx in range(0, len(events)) if idx not in invalid_indices]
        purchase_index = random.choice(pc_valid_indices) if pc_valid_indices else len(events)

        # 예외처리 : 구매 이벤트를 삽입할 위치에 view_product가 있는 경우, 구매 이벤트 위치 조정
        # ex. page_view(or search or click_promotion) → view_product 일때, page_view와 view_product 사이에 purchase가 위치하는 것 방지
        # print(f"pc_valid_indices: {pc_valid_indices}, purchase_index: {purchase_index}, length of events: {len(events)}")
        while pc_valid_indices:
            if events[purchase_index] == 'view_product':
                pc_valid_indices.remove(purchase_index)
                purchase_index = random.choice(pc_valid_indices) if pc_valid_indices else len(events)
            else:
                break

        events.insert(purchase_index, 'purchase')
        events.insert(purchase_index, 'checkout')

        if events[purchase_index-1] != 'view_product':
            events.insert(purchase_index, 'page_view')

        # 취소 이벤트 삽입 (구매 이벤트 이후 2.5% 확률로 취소 이벤트 삽입)
        cancel_prob = np.random.beta(a=1.2, b=60)
        if np.random.rand() < cancel_prob:
            purchase_index = events.index('purchase')
            cart_indices = [index for index, event in enumerate(events) if (event == 'add_to_cart') & (index > purchase_index)]

            invalid_indices = []
            invalid_indices.extend(list(range(0, purchase_index+1))) # purchase 이전 인덱스 제외
            for idx in cart_indices:
                invalid_indices.extend(list(range(idx-1, idx+1)))  # page_view → view_product → add_to_cart 사이에 위치할 수 없음

            # cancel_order 이벤트가 위치할 수 없는 인덱스 중복 제거 및 정렬
            invalid_indices = set(invalid_indices)
            invalid_indices = sorted(list(invalid_indices))

            cancel_valid_indices = [idx for idx in range(0, len(events)+1) if idx not in invalid_indices]
            
            # 예외처리 : cancel_order 이벤트가 위치할 수 없는 인덱스가 없는 경우, page_view와 cancel_order를 마지막 위치에 추가
            cancel_index = random.choice(cancel_valid_indices) if cancel_valid_indices else len(events)

            events.insert(cancel_index, 'cancel_order')
            events.insert(cancel_index, 'page_view')


    # 5. remove_from_cart 이벤트 삽입 (확률적으로)

    ## 약 15% 확률로 remove_from_cart 발생
    is_remove_from_cart = (random.random() < 0.15)

    # 구매 세션: 장바구니에 2회 이상 담았으면 일부 상품 제거 이벤트 발생 가능
    case_1 = ('purchase' in events) and (events.count('add_to_cart') >= 2)

    # 비구매 세션
    case_2 = (has_add_to_cart) and ('purchase' not in events)

    if (case_1 or case_2) and is_remove_from_cart:
        cart_indices = [index for index, event in enumerate(events) if event == 'add_to_cart']

        invalid_indices = []

        # case_1 구매 세션: invalid_indices에 add_to_cart, purchase, cancel_order 위치 추가
        if case_1:
            purchase_index = events.index('purchase')
            second_add_to_cart_index = cart_indices[1]  # 2번째 add_to_cart 이벤트 위치

            # purchase가 최초 add_to_cart의 후행일때 add_to_cart → purchase → add_to_cart
            # add_to_cart와 purchase 사이에 위치할 수 없음 (cart가 비어있을 때 purchase or remove_from_cart 발생 금지)
            if purchase_index < second_add_to_cart_index:
                invalid_indices.extend(list(range(0, second_add_to_cart_index+1)))

                for idx in cart_indices[2:]:
                    invalid_indices.extend(list(range(idx-1, idx+1)))  # page_view → view_product → add_to_cart 사이에 위치할 수 없음

            # add_to_cart → add_to_cart → purchase
            else:
                for idx in cart_indices:
                    if len(invalid_indices) == 0:
                        invalid_indices.extend(list(range(0, idx+1)))  # remove_from_cart는 최초 add_to_cart보다 선행할 수 없음
                    else:
                        invalid_indices.extend(list(range(idx-1, idx+1)))  # page_view → view_product → add_to_cart 사이에 위치할 수 없음

                invalid_indices.extend(list(range(purchase_index-1, purchase_index+1)))  # page_view(profile) → checkout → purchase 사이에 위치할 수 없음
            
            if 'cancel_order' in events:
                cancel_index = events.index('cancel_order')
                invalid_indices.append(cancel_index)  # page_view(profile) → cancel_order 사이에 위치할 수 없음

        # case_2 비구매 세션: invalid_indices에 add_to_cart 이벤트 위치 추가
        else:
            for idx in cart_indices:
                if len(invalid_indices) == 0:
                    invalid_indices.extend(list(range(0, idx+1)))  # remove_from_cart는 add_to_cart보다 선행할 수 없음
                else:
                    invalid_indices.extend(list(range(idx-1, idx+1)))  # page_view → view_product → add_to_cart 사이에 위치할 수 없음

        # purchase 이벤트가 위치할 수 없는 인덱스 중복 제거 및 정렬
        invalid_indices = set(invalid_indices)
        invalid_indices = sorted(list(invalid_indices))

        remove_valid_indices = [idx for idx in range(0, len(events)+1) if idx not in invalid_indices]

        # 예외처리 : remove_from_cart 이벤트가 위치할 수 없는 인덱스가 없는 경우, page_view와 remove_from_cart를 마지막 위치에 추가
        remove_index = random.choice(remove_valid_indices) if remove_valid_indices else len(events)

        events.insert(remove_index, 'remove_from_cart')
        events.insert(remove_index, 'page_view')

    # 6. 반품/리뷰 이벤트 삽입 (forced_event_type값이 있을 경우)
    if forced_event_type:
        # 반품 또는 리뷰 이벤트 삽입 위치 결정 (add_to_cart 바로 앞은 피함)
        add_to_cart_indices = [idx for idx, evt in enumerate(events) if evt == 'add_to_cart']
        purchase_index = events.index('purchase') if 'purchase' in events else None
        disallowed = set()
        for ac_idx in add_to_cart_indices:
            disallowed.add(ac_idx)          # add_to_cart 위치 바로 앞은 금지
            disallowed.add(ac_idx-1)        # add_to_cart 이전 인덱스도 금지
        if purchase_index is not None:
            disallowed.add(purchase_index)  # purchase 바로 앞 위치는 금지
            disallowed.add(purchase_index-1)
        # 가능한 삽입 위치 계산 (0 ~ len(events) 사이, disallowed 제외)
        possible_positions = [pos for pos in range(0, len(events)+1) if pos not in disallowed]
        insert_pos = random.choice(possible_positions) if possible_positions else len(events)
        events.insert(insert_pos, forced_event_type)
        events.insert(insert_pos, 'page_view')  # profile 페이지 방문 이벤트 추가 (반품/리뷰 이벤트 이후 방문)

    # 7. 로그인 및 회원가입 이벤트 삽입
    login_required_events = {'add_to_cart', 'remove_from_cart', 'purchase', 'logout', 'return', 'review'}
    needs_login = any(event in login_required_events for event in events)
    if needs_login:
        # 로그인 필요한 이벤트 중 첫 번째의 인덱스 찾기
        first_login_index = None
        for idx, event in enumerate(events):
            if event in login_required_events:
                first_login_index = idx
                break
        if first_login_index is None:
            first_login_index = len(events)
        # 첫 로그인 필요 이벤트보다 앞선 범위 내에서 랜덤 위치 선정
        insert_index = random.randint(1, first_login_index) if first_login_index > 0 else 0

        # 신규 유저인 경우 회원가입 이벤트를 먼저 추가
        if user_is_new:
            events.insert(insert_index, 'signup')
            insert_index += 1
        # 로그인 이벤트 추가
        events.insert(insert_index, 'login')
    else:
        # 로그인 필요 이벤트가 없지만 일부 세션은 로그인만 수행 (20% 확률, 신규면 회원가입 후 로그인)
        if random.random() < 0.2:
            insert_index = random.randint(1, len(events))
            if user_is_new:
                events.insert(insert_index, 'signup')
                events.insert(insert_index + 1, 'login')
            else:
                events.insert(insert_index, 'login')
                
    return events

In [ ]:
rng = np.random.default_rng(42)

# 재방문 간격(일) 분포 파라미터
SESSION_GAP_CFG = {
    # 충성: 지수분포(평균 5일), 꼬리 약간 허용
    'loyal':   {'dist': 'exponential', 'mean': 5.0,   'clip': (1, 45)},

    # 일반: "초기 버스트(짧은 간격)" + "안정기(긴 간격)" 혼합
    #  - burst_prob 확률로 지수(평균 7일)에서 추출 → 초반 탐색 활발
    #  - 그 외는 로그정규(평균≈25일, sigma=0.7) → 중장기 완만
    'regular': {'dist': 'mixture_lognorm',
                'burst_prob': 0.25, 'burst_mean': 7.0,
                'mean': 40.0, 'sigma': 0.70,
                'clip': (3, 120)},

    # 휴면: 지수(평균 120일)로 더 느리게, 하한 14일
    'dormant': {'dist': 'exponential', 'mean': 120.0, 'clip': (14, 600)},
}

def _sample_lognormal_days(mean, sigma, size=1, rng=rng):
    """
    로그정규의 평균을 mean으로 맞추기 위한 mu 계산: E[X] = exp(mu + 0.5*sigma^2)
    input:
        mean (float): 목표 평균(일)
        sigma (float): 로그정규 표준편차
        size (int): 샘플 개수
    return:
        np.ndarray: 일(day) 단위 샘플
    """
    mu = np.log(mean) - 0.5 * (sigma ** 2)
    return rng.lognormal(mean=mu, sigma=sigma, size=size)

def sample_session_gap_days(user_type: str, rng=rng) -> timedelta:
    """
    등급별 재방문 간격을 '연속값'으로 샘플링하여 Timedelta로 반환.
    - loyal: 지수(mean)
    - regular: 혼합(버스트: 지수 / 일반: 로그정규)
    - dormant: 지수(mean)

    기존 clip을 그대로 적용하되, '반올림'은 하지 않음(시간 단위 비교를 위해 연속값 유지).
    """
    cfg = SESSION_GAP_CFG[user_type]
    lo, hi = cfg['clip']

    if cfg['dist'] == 'exponential':
        val = rng.exponential(scale=cfg['mean'], size=1)[0]
    elif cfg['dist'] == 'lognormal':
        val = _sample_lognormal_days(cfg['mean'], cfg['sigma'], size=1, rng=rng)[0]
    elif cfg['dist'] == 'mixture_lognorm':
        if rng.random() < cfg['burst_prob']:
            val = rng.exponential(scale=cfg['burst_mean'], size=1)[0]
        else:
            val = _sample_lognormal_days(cfg['mean'], cfg['sigma'], size=1, rng=rng)[0]
    else:
        raise ValueError(f"unknown dist for {user_type}")

    # 하한/상한 클리핑 (연속값 유지)
    val = float(np.clip(val, lo, hi))
    return timedelta(days=val)

In [9]:
# ---------------------------------------
# EWMA(지수가중 이동합) 설정값
# ---------------------------------------
EWM_HALFLIFE = {
    "sessions": 45.0,   # 세션 활동의 반감기(일)
    "purchases": 60.0   # 구매 활동의 반감기(일)
}

@dataclass
class UserState:
    """
    세션 생성 중 사용자 상태(등급, 최근활동지표, 이탈 여부)를 보관하는 경량 컨테이너.

    속성:
        user_id (int): 사용자 ID
        tier (str): 현재 등급 ('loyal'|'regular'|'dormant')
        churned (bool): 이탈 여부 플래그. True면 이후 세션 생성 중단
        last_update (datetime): 최근활동(EWMA) 마지막 갱신 시각
        ewm_sessions (float): 지수가중 세션 활동 점수
        ewm_purchases (float): 지수가중 구매 활동 점수
        sessions_seen (int): 지금까지 생성된 세션 수(해당 사용자 기준)
        last_tier_eval_month (int): 마지막 등급 평가를 수행한 월(중복 평가 방지)
    """
    user_id: int
    tier: str
    churned: bool = False
    last_update: datetime = None
    ewm_sessions: float = 0.0
    ewm_purchases: float = 0.0
    sessions_seen: int = 0
    last_tier_eval_month: int = None

def _decay_factor(delta_days: float, half_life: float) -> float:
    """
    최근 활동치를 시간 경과에 따라 감쇠시키기 위한 지수감쇠 계수를 계산합니다.

    input:
        delta_days (float): 마지막 갱신 이후 경과일(일 단위)
        half_life (float): 반감기(일). 해당 일수 경과 시 값이 절반으로 감소

    return:
        float: 감쇠 계수 e^(-delta_days / half_life)
    """
    return math.exp(-(delta_days / half_life)) if delta_days > 0 else 1.0

def update_activity_ewm(state: UserState, now: datetime, session_had_purchase: bool):
    """
    세션 종료 시점에 사용자 최근활동 지표(EWMA)를 갱신합니다.

    로직:
      1) 마지막 갱신시각(last_update)와 now의 차이(Δ일)를 계산
      2) 반감기(EWM_HALFLIFE)에 따른 감쇠계수(exp(-Δ/HL))로 기존 점수 감쇠
      3) 이번 세션의 활동치를 가산:
         - 세션 1회 → ewm_sessions += 1
         - 구매 발생 시 ewm_purchases += 1
      4) last_update와 sessions_seen 업데이트

    input:
        state (UserState): 갱신 대상 사용자 상태
        now (datetime): 세션 종료 시각(보통 세션 마지막 이벤트 시간)
        session_had_purchase (bool): 세션 중 구매 발생 여부

    return:
        None (state 객체 내부 값을 갱신)
    """
    if state.last_update is None:
        state.last_update = now

    delta_days = (now - state.last_update).total_seconds() / 86400.0

    # 1) 이전 점수 감쇠
    for key, hl in EWM_HALFLIFE.items():
        factor = _decay_factor(delta_days, hl)
        if key == "sessions":
            state.ewm_sessions *= factor
        elif key == "purchases":
            state.ewm_purchases *= factor

    # 2) 이번 세션 활동 가산
    state.ewm_sessions += 1.0
    if session_had_purchase:
        state.ewm_purchases += 1.0

    # 3) 북마크 업데이트
    state.last_update = now
    state.sessions_seen += 1

In [10]:
# 등급 전환 규칙(간결/해석가능한 룰)
TIER_RULES = {
    'loyal': {
        'stay':                 {'sessions_min': 0.8, 'purchases_min': 0.15, 'p': 0.85},
        'downgrade_to_regular': {'sessions_max': 0.5, 'purchases_max': 0.05, 'p': 0.60},
        'downgrade_to_dormant': {'sessions_max': 0.2, 'purchases_max': 0.0, 'p': 0.15}
    },
    'regular': {
        'upgrade_to_loyal':     {'sessions_min': 0.9, 'purchases_min': 0.20, 'p': 0.35},
        'stay':                 {'sessions_min': 0.3, 'purchases_min': 0.05, 'p': 0.55},
        'downgrade_to_dormant': {'sessions_max': 0.2, 'purchases_max': 0.02, 'p': 0.45}
    },
    'dormant': {
        'recover_to_regular': {'sessions_min': 0.4, 'purchases_min': 0.05, 'p': 0.30},
        'stay':               {'p': 0.60}
    }
}

# 계절성 보정: 승급/복귀 확률에 월별 버프
SEASONAL_ADJ = {
    'upgrade_to_loyal': {11: 1.05, 12: 1.10},  # 연말 성수기
    'recover_to_regular': {2: 1.05, 9: 1.05}   # 시즌 전환기
}

def _seasonal_boost(key: str, month: int, p: float) -> float:
    """
    월별 계절성을 반영하여 특정 전환확률 p에 승수를 곱합니다.

    input:
        key (str): 보정 키(예: 'upgrade_to_loyal', 'recover_to_regular')
        month (int): 현재 월(1~12)
        p (float): 기본 전환 확률

    return:
        float: 보정 후 확률(최대 0.99로 캡)
    """
    if key in SEASONAL_ADJ and month in SEASONAL_ADJ[key]:
        p *= SEASONAL_ADJ[key][month]
    return min(p, 0.99)


def maybe_update_tier(state: UserState, now: datetime):
    """
    월 경계에서 1회, 등급 전환(상향/유지/하향/복귀)을 확률적으로 평가합니다.

    로직:
      1) EWMA 지표를 간단히 정규화(sessions: capped 2.0, purchases: capped 0.5)
      2) 현재 등급의 룰(TIER_RULES)을 참조하여 조건 충족 시 확률적으로 전환 시도
      3) 승급/복귀에는 계절성 보정(SEASONAL_ADJ) 적용
      4) 아무 조건에도 걸리지 않으면 기존 등급 유지

    input:
        state (UserState): 평가 대상 사용자 상태(내부 tier를 직접 갱신)
        now (datetime): 평가 시점(월 판정에 사용)

    return:
        None (state.tier를 필요 시 변경)
    """
    month = now.month
    tier = state.tier
    rules = TIER_RULES[tier]

    # 1) 간단 정규화(폭주 방지용 상한)
    s = min(state.ewm_sessions, 2.0)
    p = min(state.ewm_purchases, 0.5)

    # 2) 등급별 전환 로직
    if tier == 'loyal':
        if s <= rules['downgrade_to_dormant']['sessions_max'] and p <= rules['downgrade_to_dormant']['purchases_max']:
            if rng.random() < rules['downgrade_to_dormant']['p']:
                state.tier = 'dormant'; return
        if s <= rules['downgrade_to_regular']['sessions_max'] and p <= rules['downgrade_to_regular']['purchases_max']:
            if rng.random() < rules['downgrade_to_regular']['p']:
                state.tier = 'regular'; return
        return

    elif tier == 'regular':
        up = rules['upgrade_to_loyal']
        if (s >= up['sessions_min']) and (p >= up['purchases_min']):
            prob = _seasonal_boost('upgrade_to_loyal', month, up['p'])
            if rng.random() < prob:
                state.tier = 'loyal'; return

        down = rules['downgrade_to_dormant']
        if (s <= down['sessions_max']) and (p <= down['purchases_max']):
            if rng.random() < down['p']:
                state.tier = 'dormant'; return
        return

    else:  # dormant
        rec = rules['recover_to_regular']
        if (s >= rec['sessions_min']) and (p >= rec['purchases_min']):
            prob = _seasonal_boost('recover_to_regular', month, rec['p'])
            if rng.random() < prob:
                state.tier = 'regular'; return
        return

In [11]:
# 이탈(세션 생성 중단) 확률 설정
CHURN_CFG = {
    # 각 세션 종료 시점 hazard(기본)
    'hazard_per_session': {'loyal': 0.010, 'regular': 0.035, 'dormant': 0.120},

    # 휴면의 "첫 세션 직후" 초기 이탈 확률
    'dormant_initial_churn': 0.35,

    # 세션 수가 늘수록 hazard를 올리는 램프업(최소 변경으로 반영)
    # (threshold 세션 이상이면 factor 곱)
    'hazard_ramp': {
        'loyal':   [(5, 1.10), (10, 1.20)],     # 충성은 완만
        'regular': [(3, 1.25), (6, 1.50)],      # 일반은 중간부터 가파르게
        'dormant': [(2, 1.30), (4, 1.60)],      # 휴면은 초반부터 상승
    },

    # 상한(안정성)
    'hazard_cap': 0.95,
}

def will_churn_after_session(user_type: str, sessions_seen: int = None) -> bool:
    """
    세션 종료 시 이탈(향후 세션 생성 중단) 여부를 확률적으로 판정.
    - dormant는 첫 세션 직후 높은 초기 이탈 확률(기본 35%) 1회 평가
    - 기본 hazard에 sessions_seen 기반 램프업(hazard_ramp) 계수를 곱해 최종 확률로 베르누이 샘플링 (장기 리텐션 현실화)
    input:
        user_type: 'loyal'|'regular'|'dormant'
        is_first_session_for_user: 첫 세션 여부
        sessions_seen: 지금까지의 세션 수(현재 세션 포함 권장). None이면 램프업 미적용.
    return:
        bool: True → 이탈(이후 세션 생성 중단)
    """
    # 휴면의 첫 세션 직후 초기 이탈
    if user_type == 'dormant' and sessions_seen == 1:
        if rng.random() < CHURN_CFG['dormant_initial_churn']:
            return True

    # 기본 hazard
    hazard = CHURN_CFG['hazard_per_session'][user_type]

    # 세션 수에 따른 램프업(옵션)
    if sessions_seen is not None:
        for threshold, factor in CHURN_CFG['hazard_ramp'][user_type]:
            if sessions_seen >= threshold:
                hazard *= factor

    # 상한
    hazard = min(hazard, CHURN_CFG['hazard_cap'])
    return rng.random() < hazard

In [12]:
def generate_category_path_uris(events: pd.DataFrame, products: pd.DataFrame, events_mask, is_page_view: bool) -> pd.Series:
    """
    page_view(or search) → view_product 이벤트에 대해 카테고리 조합을 기반으로 URI를 생성하는 고성능 함수

    input:
        events: events DataFrame
        products: products DataFrame
        events_mask: 대상 이벤트 마스크 리스트
        is_page_view: page_view 이벤트 여부

    return:
        pd.Series: 생성된 URI 시리즈 (index는 원래 events와 동일)
    """
    # 1. 대상 마스크
    events_pids = events.loc[events_mask, ['product_id']].copy()

    # 2. 제품 정보 병합
    target_df = events_pids.merge(products[['id', 'department', 'category', 'sub_category', 'brand']],
                             left_on='product_id', right_on='id', how='left')

    # 3. 무작위 선택 조합 전략
    # 속성값 배열 준비
    dept = target_df['department'].values
    cat = target_df['category'].values
    sub = target_df['sub_category'].values
    brand = target_df['brand'].values

    # URI 조각 저장 배열
    length_of_target_df = len(target_df)
    uris = np.empty(length_of_target_df, dtype=object)

    # 속성 조합 패턴 생성 (속도 최적화 위해 미리 정의)
    all_patterns = [list(c) for i in range(1, 5) for c in combinations(['department', 'category', 'sub_category', 'brand'], i)]
    
    if is_page_view:
        all_patterns.remove(['sub_category'])
        all_patterns.remove(['department'])
        all_patterns.remove(['department', 'sub_category'])
        all_patterns.remove(['sub_category', 'brand'])
        all_patterns.remove(['department', 'sub_category', 'brand'])

    chosen_patterns = np.random.choice(len(all_patterns), size=length_of_target_df)

    # URI 생성
    for i in range(length_of_target_df):
        parts = []
        pattern = all_patterns[chosen_patterns[i]]
        
        if 'department' in pattern:
            parts.append(f"department/{dept[i]}")
        if 'category' in pattern:
            parts.append(f"category/{cat[i]}")
        if 'sub_category' in pattern:
            parts.append(sub[i])
        if 'brand' in pattern:
            parts.append(f"brand/{brand[i]}")

        uris[i] = "/" + "/".join(parts)

    # 반환: 원래 events 테이블에 맞춰 index 정렬
    result = pd.Series(uris, index=events[events_mask].index)
    return result


In [13]:
def generate_uri(events, products, promotions):
    """
    이벤트 유형, 다음 이벤트, 제품 정보에 기반하여 realistic한 URI 경로를 벡터 연산으로 생성합니다.

    사용자의 행동 흐름을 반영하여 각 이벤트별 서비스 내 URI 경로를 대규모 데이터셋에 빠르게 적용합니다.
    특히 page_view 이벤트 다음이 view_product일 경우 해당 상품 카테고리 경로를 생성하고, 
    그 외의 경우에는 홈, 카테고리 또는 브랜드 페이지로 연결되도록 구성합니다.

    또한 view_product, purchase, cart 관련 이벤트에는 product_id 기반의 고유 URI를 생성하며,
    search 이벤트의 경우 인기 브랜드 및 카테고리 키워드 기반으로 검색어를 생성합니다.

    input:
        events (pd.DataFrame): 사용자 이벤트 데이터프레임
                               반드시 'event_type', 'next_event_type', 'product_id' 등의 컬럼 포함
        products (pd.DataFrame): 제품 정보 데이터프레임
                                 반드시 'id', 'brand', 'department', 'category', 'sub_category' 컬럼 포함

    return:
        pd.DataFrame: URI가 포함된 사용자 이벤트 DataFrame
                      'uri' 컬럼이 추가되며, 각 이벤트의 유형에 따라 다음과 같은 형식의 경로가 생성됩니다:
                      - '/product/1234' (view_product)
                      - '/department/여성/category/상의/블라우스/brand/스파오' (page_view → view_product)
                      - '/department/남성/category/아우터' (랜덤 카테고리 page_view)
                      - '/search?q=지오다노+셔츠' (search)
                      - '/cart' (add_to_cart, remove_from_cart)
                      - '/login', '/purchase' 등 (단일 이벤트 URI)
                      - '' (click_promotion 또는 예외)
    """
    events = events.copy()

    # session_start URI
    events.loc[events['event_type'] == 'session_start', 'uri'] = '/session_start'

    # ① 단일 URI: signup, login, logout, checkout, purchase 등
    simple_map = ['signup', 'login', 'logout', 'checkout', 'purchase']
    events.loc[events['event_type'].isin(simple_map), 'uri'] = '/' + events.loc[events['event_type'].isin(simple_map), 'event_type']

    # ② view_product
    vp_mask = (events['event_type'] == 'view_product')
    events.loc[vp_mask, 'uri'] = '/product/' + events.loc[vp_mask, 'product_id'].astype(str)

    # ③-1 page_view → view_product
    pv_vp_mask = (events['event_type'] == 'page_view') & (events['next_event_type'] == 'view_product')
    pv_vp_uris = generate_category_path_uris(events, products, pv_vp_mask, is_page_view=True)
    events.loc[pv_vp_uris.index, 'uri'] = pv_vp_uris

    # ③-2 page_view → return or review
    profile_event_types = ['return', 'review', 'checkout', 'remove_from_cart', 'cancel_order']
    pv_return_review_mask = (events['event_type'] == 'page_view') & (events['next_event_type'].isin(profile_event_types))
    events.loc[pv_return_review_mask, 'uri'] = '/profile'

    # ③-3 page_view 일반
    pv_other_mask = (events['event_type'] == 'page_view') & (~events['next_event_type'].isin(profile_event_types)) & (events['next_event_type'] != 'view_product')
    pv_uris = generate_category_path_uris(events, products, pv_other_mask, is_page_view=True)
    events.loc[pv_uris.index, 'uri'] = pv_uris

    # ④ search 일반
    search_mask = (events['event_type'] == 'search') & (~events['next_event_type'].isin(['view_product', 'page_view']))
    search_uris = generate_category_path_uris(events, products, search_mask, is_page_view=False)
    events.loc[search_uris.index, 'uri'] = search_uris

    # ④-1 search → view_product
    search_vp_mask = (events['event_type'] == 'search') & (events['next_event_type'] == 'view_product')
    search_vp_uris = generate_category_path_uris(events, products, search_vp_mask, is_page_view=False)
    events.loc[search_vp_uris.index, 'uri'] = search_vp_uris

    # ④-2 search → page_view
    search_pv_mask = (events['event_type'] == 'search') & (events['next_event_type'] == 'page_view')
    search_pv_uris = generate_category_path_uris(events, products, search_pv_mask, is_page_view=False)
    events.loc[search_pv_uris.index, 'uri'] = search_pv_uris

    # ⑤ 장바구니 관련
    cart_mask = events['event_type'].isin(['add_to_cart', 'remove_from_cart'])
    events.loc[cart_mask, 'uri'] = '/cart'

    # ⑥ 반품 또는 리뷰 작성 이벤트
    return_review_mask = events['event_type'].isin(['return', 'review'])
    events.loc[return_review_mask, 'uri'] = '/' + events.loc[return_review_mask, "event_type"]

    # ⑦ 프로모션 관련
    promotion_mask = events['event_type'].isin(['click_promotion'])
    events.loc[promotion_mask, 'uri'] = '/timesale'

    return events

In [ ]:
def update_signup_date(users, events):
    """
    신규 고객의 가입일(users.created_at)을 events 테이블에서 발생한 signup 이벤트 시점으로 동기화하는 함수

    input:
        users (pd.DataFrame): 사용자 정보 테이블
                              반드시 'id', 'created_at' 컬럼 포함
        events (pd.DataFrame): 사용자 이벤트 테이블
                               반드시 'user_id', 'event_type', 'created_at' 컬럼 포함

    return:
        pd.DataFrame: 가입일이 갱신된 users 테이블
    """
    updated_signup_dates = (
        events.loc[events['event_type'] == 'signup', ['user_id', 'created_at']]
        .set_index('user_id')
        .to_dict()['created_at']
    )

    users['created_at'] = users.apply(
        lambda x: updated_signup_dates[x['id']] if x['id'] in updated_signup_dates else x['created_at'],
        axis=1
    )

    print(f"Number of users with updated signup date: {len(updated_signup_dates)}")
    
    return users

## 데이터 생성

In [ ]:
# Seed & Faker 설정
random.seed(42)
np.random.seed(42)
faker = Faker("ko_KR")

# 데이터 로드
users_df = pd.read_csv("./data/users.csv", parse_dates=['birth', 'created_at'])
visitor_df = pd.read_csv("./data/visitor_stats_by_city_month.csv")
promotion_df = pd.read_csv("./data/promotions.csv", parse_dates=['start_date', 'end_date'])

# 시뮬레이션 설정
start_date = datetime(2022, 1, 1)
end_date = datetime(2025, 1, 1)

# 사용자 유형 분포 설정 및 유형에 따른 세션 생성 간격(방문 간격) 설정
num_users = users_df.shape[0]
users_df['user_type'] = random.choices(['loyal', 'regular', 'dormant'], weights=[0.05, 0.10, 0.85], k=num_users)

# 메타 데이터 설정
BROWSERS_PC = ["Chrome", "Edge", "Whale", "Safari"]
BROWSERS_MOBILE = ["Chrome", "Edge", "Safari", "Samsung Internet"]

EVENT_TYPES = [
    "session_start", "signup", "login", "logout", "view_product", "page_view",
    "scroll", "search", "click_promotion", "add_to_cart", "remove_from_cart",
    "checkout", "purchase"
]

# 확률 범위 설정 (디바이스, 브라우저, 트래픽 소스)
device_range = {"Mobile": (0.75, 0.85), "PC": (0.15, 0.25)}
browser_range = {
    "Chrome": (0.5, 0.6), "Safari": (0.18, 0.25), "Samsung Internet": (0.1, 0.15),
    "Edge": (0.02, 0.05), "Whale": (0.01, 0.03)
}
traffic_range_base = {
    "direct": (0.15, 0.25), "organic_search": (0.35, 0.45), "paid_search": (0.05, 0.1),
    "facebook": (0.05, 0.1), "instagram": (0.05, 0.1), "kakao": (0.03, 0.07),
    "email": (0.01, 0.03), "referral": (0.02, 0.05)
}

six_hours = timedelta(hours=6)
fifteen_days = timedelta(days=15)

events = []
states = {}  # user_id -> UserState
event_id = 1
new_users_signed = set()
namespace = uuid.NAMESPACE_DNS

# 고객 행동 데이터 생성
for idx, row in users_df.iterrows():
    user_id = row["id"]
    user_type = row["user_type"]
    signup_date = pd.to_datetime(row["created_at"])
    is_new_user_overall = (signup_date >= start_date)
    num_sessions = 0  # 고객별 생성된 세션의 수 초기화

    # 고객 상태 초기화
    states[user_id] = UserState(user_id=user_id, tier=user_type, last_update=None)

    # 첫 세션 시작 시간 설정
    if is_new_user_overall:
        # 신규 유저: 가입 시점 직전에 첫 세션 시작
        session_time = signup_date - timedelta(seconds=random.randint(1, 600))
    else:
        # 기존 유저: 시뮬레이션 시작 후 첫 한 달 내의 랜덤 시점에서 세션 시작
        session_time = start_date + timedelta(days=random.randint(0, 30))

    # 반품 또는 리뷰 작성 이벤트 생성 여부 초기화
    inserted_type = None

    # 유저별 여러 세션 생성
    while session_time < end_date:
        session_id = 'e-' + str(uuid.uuid5(namespace, str(event_id)))[:12]
        
        state = states[user_id]

        # 고객이 이탈했으면 세션 생성 중단
        if state.churned:
            break

        # 세션 시작 시간 설정
        current_time = session_time

        # 세션별 디바이스, 브라우저, 트래픽 소스 확률 분포 생성 및 선택
        device_probs = generate_prob_from_range(device_range)
        browser_probs = generate_prob_from_range(browser_range)
        base_traffic_probs = generate_prob_from_range(traffic_range_base)
        traffic_probs = adjust_traffic_for_month(base_traffic_probs, session_time.month)

        device = np.random.choice(list(device_probs.keys()), p=list(device_probs.values()))
        traffic_source = np.random.choice(list(traffic_probs.keys()), p=list(traffic_probs.values()))
        
        possible_browsers = BROWSERS_MOBILE if device == "Mobile" else BROWSERS_PC
        possible_browsers = [b for b in possible_browsers if b in browser_probs]
        browser_weights = np.array([browser_probs[b] for b in possible_browsers])
        browser_weights /= browser_weights.sum()
        browser = np.random.choice(possible_browsers, p=browser_weights)

        ip_address = faker.ipv4_public()
        if random.random() < 0.7:
            address = row['address']
        else:
            address = generate_address_by_visitor_stats(current_time, visitor_df) 

        # 세션 내 이벤트 시퀀스 생성
        ## 신규 유저 여부 판단
        user_is_new_for_session = False
        if is_new_user_overall and user_id not in new_users_signed:
            user_is_new_for_session = True
            new_users_signed.add(user_id)

        # 세션 내 이벤트 목록 생성
        session_events = generate_session_events(session_id, user_id, user_is_new_for_session, traffic_source, current_time, promotion_df, forced_event_type=inserted_type)

        # 세션 이벤트들 추가 (session_start 이벤트 + 생성된 session_events 목록)
        # 세션의 시작 이벤트 session_start 삽입
        sequence = 1

        events.append({
            "id": event_id,
            "user_id": user_id,
            "session_id": session_id,
            "sequence": sequence,
            "event_type": "session_start",
            "created_at": current_time,
            "device": device,
            "browser": browser,
            "traffic_source": traffic_source,
            "ip_address": ip_address,
            "address": address
        })

        event_id += 1
        prev_event = "session_start"

        # 나머지 세션 이벤트 처리
        for event_type in session_events:
            # 이전 이벤트 대비 시간 간격 계산
            gap = event_time_gap(prev_event, event_type)
            current_time += timedelta(seconds=float(gap))

            sequence += 1

            event_data = {
                "id": event_id,
                "user_id": user_id,
                "session_id": session_id,
                "sequence": sequence,
                "event_type": event_type,
                "created_at": current_time,
                "device": device,
                "browser": browser,
                "traffic_source": traffic_source,
                "ip_address": ip_address,
                "address": address
            }

            event_id += 1
            events.append(event_data)
            prev_event = event_type

        # --------------- 세션 종료 후: 활동지표 갱신 → 등급 평가(월 1회) → 이탈 판정 --------------- #
        # 유저별 생성된 세션 수 증가
        num_sessions += 1
        
        # 방금 생성한 세션에 구매 발생 여부
        has_purchase_session = ('purchase' in session_events) & ('cancel_order' not in session_events)
        
        # 세션 종료 시각
        session_end_time = current_time
        
        # 활동지표 갱신
        update_activity_ewm(state, now=session_end_time, session_had_purchase=has_purchase_session)
        
        # 등급 평가
        maybe_update_tier(state, now=session_end_time)

        # 이탈 여부 판정
        if will_churn_after_session(state.tier, sessions_seen=num_sessions):
            state.churned = True
            break

        # 다음 세션까지의 간격 결정
        gap_days = sample_session_gap_days(state.tier)

        # --------------------- return or review 이벤트 생성 여부 설정 --------------------- #

        # 이전 세션에서 구매가 발생했을때 일정 확률에 따라 return or review 이벤트 생성
        if has_purchase_session:
            rand_val = random.random()
            if rand_val < 0.10:
                inserted_type = 'return'
            elif rand_val < 0.13:
                inserted_type = 'review'
            else:
                inserted_type = None
        else:
            inserted_type = None
        
        limit_date = pd.Timestamp(session_time + fifteen_days).normalize().to_pydatetime()  # return or review가 발생할 수 있는 제한 시간(세션 생성일부터 15일 후 00시까지)

        # 다음 세션 생성 시간 계산
        session_time = current_time + gap_days
        
        is_possible_return_or_review = (session_time < limit_date)  # 세션 생성일부터 15일 이내 인 경우

        if gap_days < six_hours:
            inserted_type = None
            continue

        # ----------------------- return or review 이벤트 생성 로직 ----------------------- #

        # 이전 세션에서 구매가 발생했을 경우 일정 확률에 따라 다음 세션에서 반품 또는 리뷰 작성 이벤트 발생
        ## inserted_type이 있고, 다음 세션 간격이 15일 초과인 경우
        ### 이전 세션과 다음 세션 사이(이전 세션 발생 후 15일 이내)에 반품 또는 리뷰 작성 세션 추가 (세션 간격 gap_days가 15일 이내일 경우, 다음 세션에서 반품 또는 리뷰 작성 이벤트 생성)
        if inserted_type and not is_possible_return_or_review:
            # 반품 또는 리뷰 작성 이벤트 생성
            ins_current_time = current_time + timedelta(days=random.randint(1, 14))
            ins_session_time = ins_current_time
            
            # 세션 생성 시간이 시뮬레이션 종료 시간 이전인 경우에만 세션 생성
            if ins_current_time < end_date:
                ins_session_id = 'e-' + str(uuid.uuid5(namespace, str(event_id)))[:12]
                user_is_new_for_ins = False

                # 새 세션의 디바이스, 브라우저, 트래픽 소스 무작위 선택
                device = np.random.choice(list(device_probs.keys()), p=list(device_probs.values()))
                traffic_source = np.random.choice(list(traffic_probs.keys()), p=list(traffic_probs.values()))

                possible_browsers = BROWSERS_MOBILE if device == "Mobile" else BROWSERS_PC
                possible_browsers = [b for b in possible_browsers if b in browser_probs]
                browser_weights = np.array([browser_probs[b] for b in possible_browsers])
                browser_weights /= browser_weights.sum()
                browser = np.random.choice(possible_browsers, p=browser_weights)

                ip_address = faker.ipv4_public()
                if random.random() < 0.7:
                    address = row['address']
                else:
                    address = generate_address_by_visitor_stats(ins_current_time, visitor_df) 

                # 강제 이벤트 타입을 지정하여 세션 이벤트 생성
                inserted_events = generate_session_events(ins_session_id, user_id, user_is_new_for_ins, traffic_source, ins_current_time, promotion_df, forced_event_type=inserted_type)     

                # 세션 이벤트들 추가 (session_start 이벤트 + 생성된 session_events 목록)
                # 세션의 시작 이벤트 session_start 삽입
                sequence = 1

                events.append({
                    "id": event_id,
                    "user_id": user_id,
                    "session_id": ins_session_id,
                    "sequence": sequence,
                    "event_type": "session_start",
                    "created_at": ins_current_time,
                    "device": device,
                    "browser": browser,
                    "traffic_source": traffic_source,
                    "ip_address": ip_address,
                    "address": address
                })

                event_id += 1
                prev_event = "session_start"

                # 나머지 세션 이벤트 처리
                for event_type in inserted_events:
                    # 이전 이벤트 대비 시간 간격 계산
                    gap = event_time_gap(prev_event, event_type)
                    ins_current_time += timedelta(seconds=float(gap))

                    sequence += 1
                    
                    event_data = {
                        "id": event_id,
                        "user_id": user_id,
                        "session_id": ins_session_id,
                        "sequence": sequence,
                        "event_type": event_type,
                        "created_at": ins_current_time,
                        "device": device,
                        "browser": browser,
                        "traffic_source": traffic_source,
                        "ip_address": ip_address,
                        "address": address
                    }

                    event_id += 1
                    events.append(event_data)
                    prev_event = event_type

                # 유저별 생성된 세션 수 증가
                num_sessions += 1

                # 세션 종료 시각
                ins_session_end_time = ins_current_time

                # 방금 생성한 세션에 구매 발생 여부
                has_purchase_session = ('purchase' in inserted_events) & ('cancel_order' not in inserted_events)
                
                # 활동지표 갱신
                update_activity_ewm(state, now=ins_session_end_time, session_had_purchase=has_purchase_session)

                # 이탈 여부 판정
                if will_churn_after_session(state.tier, sessions_seen=num_sessions):
                    state.churned = True
                    break

                # 이전 세션에서 구매가 발생했을때 및 일정 확률에 따라 return or review 이벤트 생성
                gap_end_to_start = (session_time - ins_session_end_time)  # 다음 세션의 시작 시간과 이전 세션의 종료 시간 간격
                gap_start_to_start = (session_time - ins_session_time)  # 다음 세션의 시작 시간과 이전 세션의 시작 시간 간격

                is_possible_return_or_review = (gap_end_to_start > six_hours) & (gap_start_to_start < fifteen_days)

                if has_purchase_session and is_possible_return_or_review:
                    rand_val = random.random()
                    if rand_val < 0.10:
                        inserted_type = 'return'
                    elif rand_val < 0.13:
                        inserted_type = 'review'
                    else:
                        inserted_type = None
                else:
                    inserted_type = None

        # -------------------------------------------------------------------------------- #


# 결과 DataFrame 생성
events_df = pd.DataFrame(events)

print(events_df.info())
events_df.head()

## URI 생성

In [ ]:
products_df = pd.read_csv("./data/products.csv")
product_ids = products_df['id'].tolist()

# 1. search or page_view → view_product uri 생성을 위한 product_id 처리
## 1-1. product_id 랜덤 생성 (view_product에만)
events_df = events_df.sort_values(["user_id", "session_id", "created_at"]).reset_index(drop=True)
events_df["next_event_type"] = events_df.groupby(["user_id", "session_id"])["event_type"].shift(-1)

view_product_mask = (events_df["event_type"] == "view_product")
events_df.loc[view_product_mask, "product_id"] = np.random.choice(product_ids, size=view_product_mask.sum())

## 1-2. search or page_view → view_product 흐름에서 product_id 복사
search_pv_vp_mask = (events_df["event_type"].isin(["page_view", "search"])) & (events_df["next_event_type"] == "view_product")
vp_indices = events_df[search_pv_vp_mask].index
vp_next_indices = vp_indices + 1

# 1-3. next row가 view_product인지 확인 (안정성 보강)
valid_vp_idx = vp_next_indices[vp_next_indices < len(events_df)]
valid_search_pv_idx = vp_indices[vp_next_indices < len(events_df)]

## 1-4. product_id 복사
events_df.loc[valid_search_pv_idx, "product_id"] = events_df.loc[valid_vp_idx, "product_id"].values


# 2. search → page_view uri 생성을 위한 product_id 처리
## 2-1. search → page_view 마스크
search_to_pv_mask = (events_df["event_type"] == "search") & (events_df["next_event_type"] == "page_view")

search_indices = events_df[search_to_pv_mask].index
pv_indices = search_indices + 1

# 인덱스 범위 내 유효성 필터링
valid_idx_mask = pv_indices < len(events_df)
valid_search_idx = search_indices[valid_idx_mask]
valid_pv_idx = pv_indices[valid_idx_mask]

## 2-2. page_view의 product_id가 없는 경우 → page_view에 랜덤 생성
no_product_id_mask = events_df.loc[valid_pv_idx, "product_id"].isna()
missing_pv_idx = valid_pv_idx[no_product_id_mask]

events_df.loc[missing_pv_idx, "product_id"] = np.random.choice(
    product_ids, size=len(missing_pv_idx)
)

## 2-3. page_view의 product_id → search에 복사
events_df.loc[valid_search_idx, "product_id"] = events_df.loc[valid_pv_idx, "product_id"].values

# 3. 그 외 search의 uri 생성을 위한 product_id 처리
search_no_pid_mask = (events_df["event_type"] == "search") & (events_df["product_id"].isna())
events_df.loc[search_no_pid_mask, "product_id"] = np.random.choice(
    product_ids, size=search_no_pid_mask.sum()
)

# 4. 그 외 page_view의 uri 생성을 위한 product_id 처리
pv_no_pid_mask = (events_df["event_type"] == "page_view") & (events_df["product_id"].isna())
events_df.loc[pv_no_pid_mask, "product_id"] = np.random.choice(
    product_ids, size=pv_no_pid_mask.sum()
)

In [ ]:
events_df = generate_uri(events_df, products_df, promotion_df)

In [ ]:
events_df.drop(columns=["next_event_type", "product_id"], inplace=True)

events_df['created_at'] = events_df['created_at'].dt.strftime('%Y-%m-%d %H:%M:%S')
events_df['created_at'] = pd.to_datetime(events_df['created_at'])

In [ ]:
over_end_date_session_ids = events_df.loc[events_df['created_at'] > end_date, "session_id"].unique()
events_df = events_df[~events_df['session_id'].isin(over_end_date_session_ids)]

In [114]:
events_df = events_df.sort_values(by=['id']).reset_index(drop=True)
events_df.to_csv("./data/events.csv", index=False)

## users 신규 고객의 가입일 갱신(events에서 signup 이벤트가 발생한 시점)

In [ ]:
if 'users_df' in globals():
    
    if "user_type" in users_df.columns:
        users_df = users_df.drop(columns=['user_type'])
else:
    users_df = pd.read_csv("./data/users.csv", parse_dates=['birth', 'created_at'])

users_df = update_signup_date(users_df, events_df)

In [ ]:
users_df.to_csv('./data/users.csv', index=False)

# Order_items

In [ ]:
# 1. events_cart_df 생성 

# seed 설정
random.seed(42)
np.random.seed(42)

# 데이터 로드 
events_df = pd.read_csv("./data/events.csv", parse_dates=["created_at"])

# Step 1: 주요 이벤트(add_to_cart, purchase, click_promotion, view_product)만 추출
filtered_df = events_df[events_df['event_type'].isin(['add_to_cart', 'purchase', 'click_promotion', 'view_product'])].copy()
filtered_df['product_id'] = filtered_df["uri"].str.extract(r'/product/([^/]+)$')

# 이전 이벤트 정보 추가
filtered_df['prev_event_type'] = filtered_df.groupby("user_id")["event_type"].shift(1)
filtered_df['prev_product_id'] = filtered_df.groupby("user_id")["product_id"].shift(1)
filtered_df['prev2_event_type'] = filtered_df.groupby("user_id")["event_type"].shift(2)


# 장바구니/구매 이벤트(add_to_cart, purchase) 필터링 및 프로모션 여부 판단
cart_df = filtered_df[filtered_df['event_type'].isin(['add_to_cart', 'purchase'])].copy()
cart_df['is_promotion'] = filtered_df['prev2_event_type'] == 'click_promotion'

# 컬럼 정리
keep_cols = ['id', 'user_id', 'session_id', 'event_type',
             'created_at', 'prev_event_type', 'prev_product_id', 'is_promotion']
cart_df = cart_df[keep_cols]

# Step 2: remove_from_cart 처리
remove_df = events_df[events_df['event_type'] == 'remove_from_cart'].copy()
remove_df['prev_event_type'] = None
remove_df['prev_product_id'] = None
remove_df['is_promotion'] = False

remove_df = remove_df[keep_cols]

# Step 3: 테이블 병합 및 정렬 
events_cart_df = pd.concat([cart_df, remove_df], axis=0)
events_cart_df = events_cart_df.sort_values(['user_id', 'created_at']).reset_index(drop=True)

# 다음 이벤트 정보 추가 
events_cart_df['next_event_type'] = events_cart_df.groupby("user_id")["event_type"].shift(-1)

# Step 4: 장바구니 및 구매 컬럼 생성
events_cart_df['cart'] = None
events_cart_df['purchased_products'] = None

# Step 5: 사용자별 장바구니 및 구매 처리
user_groups = events_cart_df.groupby('user_id', sort=False).groups  


empty_cart_remove_indices = [] # 장바구니가 비었을 때 remove_from_cart 이벤트 인덱스

empty_cart_purchase_indices = []  # 장바구니가 비었을 때 purchase 이벤트 인덱스


for user_id, indices in user_groups.items():
    cart_state = {'general': [], 'timesale': []}

    for idx in indices:
        row = events_cart_df.loc[idx]
        event = row['event_type']
        product = row['prev_product_id']
        is_promo = row['is_promotion']
        prev_event = row['prev_event_type']
        next_event = row['next_event_type']

        if event == 'add_to_cart':
            key = 'timesale' if is_promo else 'general'
            cart_state[key].append(product)
            events_cart_df.at[idx, 'cart'] = copy.deepcopy(cart_state)

        elif event == 'remove_from_cart':
            combined = cart_state['general'] + cart_state['timesale']
        
            if len(combined) > 0:  # 빈 장바구니 방지
                if next_event == 'purchase' and len(combined) > 1:
                    max_removable = len(combined) - 1 # 다음 이벤트가 purchase면 최소 1개 남기기
                    remove_n = random.randint(1, max_removable)
                else: 
                    remove_n = random.randint(1, len(combined)) # 이외 이벤트는 전부 제거 가능
        
                to_remove = random.sample(combined, k=remove_n)
                cart_state['general'] = [p for p in cart_state['general'] if p not in to_remove]
                cart_state['timesale'] = [p for p in cart_state['timesale'] if p not in to_remove]
        
            else:
                empty_cart_remove_indices.append(idx)  # 빈 장바구니였던 경우 기록
        
            events_cart_df.at[idx, 'cart'] = copy.deepcopy(cart_state)

        elif event == 'purchase':
            purchased = {'general': [], 'timesale': []}
            if prev_event == 'view_product':
                # 직접 구매 상품 
                if is_promo:
                    purchased['timesale'] = [product]
                else:
                    purchased['general'] = [product]
            else:
                # 장바구니 기반 구매
                combined = cart_state['general'] + cart_state['timesale']
                if combined:                                                   
                    if next_event == 'remove_from_cart' and len(combined) > 1: # 다음 이벤트가 remove_from_cart면 최소 1개 남기고 구매
                        max_purchase = len(combined) - 1  
                        purchase_n = random.randint(1, max_purchase)
                    else:                                                         
                        purchase_n = random.randint(1, len(combined)) # 이외 이벤트는 1 ~ 전체 구매

                    to_purchase = random.sample(combined, k=purchase_n)
                    purchased['general'] = [p for p in to_purchase if p in cart_state['general']]
                    purchased['timesale'] = [p for p in to_purchase if p in cart_state['timesale']]

                    # 장바구니에서 구매된 상품 제거
                    cart_state['general'] = [p for p in cart_state['general'] if p not in purchased['general']]
                    cart_state['timesale'] = [p for p in cart_state['timesale'] if p not in purchased['timesale']]
                else:
                    empty_cart_purchase_indices.append(idx)  # 빈 장바구니였던 경우 기록
                    

            events_cart_df.at[idx, 'purchased_products'] = purchased
            events_cart_df.at[idx, 'cart'] = copy.deepcopy(cart_state)

In [ ]:
# 2 


# Step 1. purchased_products 컬럼 기준으로 구매 상품 분해

# 구매 정보가 있는 행만 필터링
events_cart_df = events_cart_df.dropna(subset=['purchased_products']).copy()

# 일반 상품 (general) 추출
general_df = events_cart_df[['user_id', 'id', 'session_id', 'event_type', 'created_at']].copy()
general_df['product_id'] = events_cart_df['purchased_products'].apply(lambda x: x.get('general', []))
general_df = general_df[general_df['product_id'].apply(lambda x: len(x) > 0)]
general_df['general'] = True
general_df['timesale'] = False
general_df = general_df.explode('product_id')

# 타임세일 상품 (timesale) 추출
timesale_df = events_cart_df[['user_id', 'id', 'session_id', 'event_type', 'created_at']].copy()
timesale_df['product_id'] = events_cart_df['purchased_products'].apply(lambda x: x.get('timesale', []))
timesale_df = timesale_df[timesale_df['product_id'].apply(lambda x: len(x) > 0)]
timesale_df['general'] = False
timesale_df['timesale'] = True
timesale_df = timesale_df.explode('product_id')

# general / timesale 구매 상품 병합
purchased_df = pd.concat([general_df, timesale_df], ignore_index=True)
purchased_df = purchased_df.dropna(subset=['product_id'])


# Step 2. return, cancel 이벤트 추출 및 복사

# 필요한 이벤트만 복사
events_copy_df = events_df[['id', 'user_id', 'session_id', 'event_type', 'created_at']].copy()
return_df = events_copy_df[events_copy_df['event_type'] == 'return'].copy()
cancel_df = events_copy_df[events_copy_df['event_type'] == 'cancel_order'].copy()

# purchase/return 타임스탬프 컬럼 추가
purchase_df = purchased_df[purchased_df['event_type'] == 'purchase'].copy()
purchase_df['created_at_purchase'] = purchase_df['created_at']
return_df['created_at_return'] = return_df['created_at']


return_matched = []
for user_id, return_group in return_df.groupby("user_id"):
    if user_id not in purchase_df['user_id'].values:
        continue

    # 해당 유저의 purchase 데이터 정렬
    user_purchase_df = purchase_df[purchase_df['user_id'] == user_id].sort_values('created_at_purchase')

    for _, return_row in return_group.iterrows():
        return_time = return_row['created_at_return']

        # 조건: return 발생 기준 1~15일 전 사이의 purchase 모두 포함
        start_time = (return_time - timedelta(days=15)).normalize()  # 15일 전 자정
        end_time = return_time - timedelta(hours=6)                   # return 이벤트 생성 시점 6시간 전 

        # 해당 범위에 있는 모든 purchase
        valid_purchases = user_purchase_df[
            (user_purchase_df['created_at_purchase'] >= start_time) &
            (user_purchase_df['created_at_purchase'] <= end_time)
        ]
        # 여러 purchase에 동일한 return_row를 복제하여 연결
        for _, purchase_row in valid_purchases.iterrows():
            merged_row = return_row.to_dict()
            for col in ['id', 'event_type', 'created_at', 'product_id']:
                merged_row[f'{col}_purchase'] = purchase_row.get(col)
            return_matched.append(merged_row)

# 결과 DataFrame으로 변환
return_link_df = pd.DataFrame(return_matched)

# purchase_df에 return 정보 병합
purchase_return_df = pd.merge(purchase_df, return_link_df, how='left', left_on=['id', 'product_id'], right_on=['id_purchase', 'product_id_purchase'])

# 컬럼 정리
columns = ['user_id_x', 'id_x', 'event_type_x', 'created_at_x', 'product_id',
           'general', 'timesale', 'id_y', 'event_type_y', 'created_at_return']
purchase_return_df = purchase_return_df[columns]

# 컬럼명 정리
purchase_return_df = purchase_return_df.rename(columns={
    'user_id_x': 'user_id',
    'id_x': 'id_purchase',
    'event_type_x': 'event_type_purchase',
    'created_at_x': 'created_at_purchase',
    'id_y': 'id_return',
    'event_type_y': 'event_type_return'
})


# Step 4. cancel - purchase 연결 (merge_asof)

# cancel 이벤트에 타임스탬프 복사
cancel_df['created_at_cancel'] = cancel_df['created_at']

cancel_matched = []
for user_id, cancel_group in cancel_df.groupby("user_id"):
    if user_id not in purchase_return_df['user_id'].values:
        continue

    # 유저별 purchase 정렬 후 merge_asof
    user_purchase_df = purchase_df[purchase_df['user_id'] == user_id].copy()
    merged_df = pd.merge_asof(
        cancel_group.sort_values('created_at_cancel'),
        user_purchase_df.sort_values('created_at_purchase'),
        by='user_id',
        left_on='created_at_cancel',
        right_on='created_at_purchase',
        direction='backward'
    )
    cancel_matched.append(merged_df)

# cancel → purchase 매핑 결과 DataFrame 생성
cancel_link_df = pd.concat(cancel_matched, ignore_index=True)
cancel_link_df = cancel_link_df[['id_y', 'id_x', 'event_type_x', 'created_at_cancel']]
cancel_link_df = cancel_link_df.rename(columns={
    'id_y': 'id_purchase',
    'id_x': 'id_cancel',
    'event_type_x': 'event_type_cancel'
})

# purchase_return_df에 cancel 정보 병합
purchase_return_cancel_df = pd.merge(purchase_return_df, cancel_link_df, how='left', on='id_purchase')


# Step 5. order_items_df 생성
order_items_df = purchase_return_cancel_df.copy()

In [ ]:
# 3 


# Step 1. 상태 및 시간 컬럼 초기화
# status, shipped_at, delivered_at, returned_at 생성 
order_items_df['status'] = None
order_items_df['shipped_at'] = pd.NaT
order_items_df['delivered_at'] = pd.NaT
order_items_df['returned_at'] = pd.NaT
end_date = datetime(2025, 1, 1)


# Step 2. 상태 (status) 설정
# Returned 상태 무작위 지정 (반품 이벤트 그룹 중 일부만 선택적으로 Returned)
returned_mask = order_items_df['event_type_return'].notna()
for _, group_df in order_items_df.loc[returned_mask].groupby('id_return'):
    if random.random() < 0.5:
        order_items_df.loc[group_df.index, 'status'] = 'Returned'
    else:
        chosen_idx = random.choice(group_df.index.tolist())
        order_items_df.loc[chosen_idx, 'status'] = 'Returned'

# Cancelled 상태 지정
order_items_df.loc[order_items_df['event_type_cancel'].notna(), 'status'] = 'Cancelled'


# Step 3. 주문 생성 기준 정보 설정

# 주문 생성 기준 이벤트 ID 설정
mask_return = order_items_df['status'] == 'Returned'
mask_cancel = order_items_df['status'] == 'Cancelled'

order_items_df['created_from_event_id'] = order_items_df['id_purchase']
order_items_df.loc[mask_return, 'created_from_event_id'] = order_items_df.loc[mask_return, 'id_return']
order_items_df.loc[mask_cancel, 'created_from_event_id'] = order_items_df.loc[mask_cancel, 'id_cancel']

# 주문 생성 시점(created_at) 설정
order_items_df['created_at'] = order_items_df['created_at_purchase']
order_items_df.loc[mask_return, 'created_at'] = order_items_df.loc[mask_return, 'created_at_return']
order_items_df.loc[mask_cancel, 'created_at'] = order_items_df.loc[mask_cancel, 'created_at_cancel']


# Step 4. 배송 및 반품 시간 생성

# created_from_event_id 단위로 그룹화하여 처리
for order, group_df in order_items_df.groupby('created_from_event_id'):

    # Step 4 내부, Returned 상태 처리 영역에서 이 부분을 교체
    # returned_idx = group_df[group_df['status'] == 'Returned'].index
    
    returned_idx = group_df[group_df['status'] == 'Returned'].index
    
    # shipped_at 초기값 설정 (모든 Returned 에 대해 일단 고정값 6시간)
    order_items_df.loc[returned_idx, 'shipped_at'] = order_items_df.loc[returned_idx, 'created_at_purchase'] + pd.Timedelta(hours=6)
    
    # 조건 분기: 반품 생성 시점이 배송 전일 경우 → 배송 시점은 고정 (6시간 후), 이후 배송/반품 시간 생성
    early_return_mask = order_items_df.loc[returned_idx, 'created_at_return'] < order_items_df.loc[returned_idx, 'shipped_at']
    early_idx = returned_idx[early_return_mask]
    
    # → 고정 배송, 고정 이후 시간 설정
    order_items_df.loc[early_idx, 'delivered_at'] = order_items_df.loc[early_idx, 'shipped_at'] + pd.to_timedelta(np.random.uniform(24, 72, len(early_idx)), unit='h')
    order_items_df.loc[early_idx, 'returned_at'] = order_items_df.loc[early_idx, 'delivered_at'] + pd.to_timedelta(np.random.uniform(24, 72, len(early_idx)), unit='h')
    
    # 반대의 경우 (정상 흐름) → 기존 로직 유지
    late_idx = returned_idx[~early_return_mask]
    
    order_items_df.loc[late_idx, 'shipped_at'] = order_items_df.loc[late_idx, 'created_at_purchase'] + pd.to_timedelta(np.random.uniform(6, 24, len(late_idx)), unit='h')
    order_items_df.loc[late_idx, 'delivered_at'] = order_items_df.loc[late_idx, 'shipped_at'] + pd.to_timedelta(np.random.uniform(24, 72, len(late_idx)), unit='h')
    order_items_df.loc[late_idx, 'returned_at'] = order_items_df.loc[late_idx, 'created_at_return'] + pd.to_timedelta(np.random.uniform(24, 72, len(late_idx)), unit='h')
    
    # 보정: created_at_return ≤ delivered_at 인 경우 returned_at 재설정
    mask = (order_items_df.loc[late_idx, 'created_at_return'] <= order_items_df.loc[late_idx, 'delivered_at'])
    idx_mask = late_idx[mask]
    order_items_df.loc[idx_mask, 'returned_at'] = order_items_df.loc[idx_mask, 'delivered_at'] + pd.to_timedelta(np.random.uniform(24, 72, len(idx_mask)), unit='h')
    
    # end_date 초과 보정
    order_items_df.loc[returned_idx[order_items_df.loc[returned_idx, 'returned_at'] >= end_date], 'returned_at'] = pd.NaT
    order_items_df.loc[returned_idx[order_items_df.loc[returned_idx, 'delivered_at'] >= end_date], 'delivered_at'] = pd.NaT

    # 기타 상태 (None) 배송 시간 생성 및 상태 결정
    default_idx = group_df[group_df['status'].isna()].index

    order_items_df.loc[default_idx, 'shipped_at'] = order_items_df.loc[default_idx, 'created_at_purchase'] + pd.to_timedelta(np.random.uniform(6, 24, len(default_idx)), unit='h')
    order_items_df.loc[default_idx, 'delivered_at'] = order_items_df.loc[default_idx, 'shipped_at'] + pd.to_timedelta(np.random.uniform(24, 72, len(default_idx)), unit='h')

    # 배송 시점에 따라 상태 결정
    mask_processing = default_idx[order_items_df.loc[default_idx, 'shipped_at'] >= end_date]
    order_items_df.loc[mask_processing, ['status', 'shipped_at', 'delivered_at']] = ['Packing', pd.NaT, pd.NaT]

    mask_shipped = default_idx[(order_items_df.loc[default_idx, 'shipped_at'] < end_date) &
                               (order_items_df.loc[default_idx, 'delivered_at'] >= end_date)]
    order_items_df.loc[mask_shipped, ['status', 'delivered_at']] = ['Shipped', pd.NaT]

    mask_purchased = default_idx[order_items_df.loc[default_idx, 'delivered_at'] < end_date]
    order_items_df.loc[mask_purchased, 'status'] = 'Purchased'


# Step 5. 주문 항목 ID 및 주문 ID 생성

# 주문 항목 ID 생성 (고정 prefix + UUID 해시)
namespace = uuid.NAMESPACE_DNS
order_items_df['id'] = [str(uuid.uuid4())[:12] for _ in range(len(order_items_df))]
order_items_df['id'] = ['oi-' + str(uuid.uuid5(namespace, str(idx)))[:12] for idx in range(len(order_items_df))]

# id_purchase 컬럼을 활용해 order_id 생성
namespace = uuid.NAMESPACE_DNS
order_items_df['order_id'] = [f"or-{str(uuid.uuid5(namespace, str(event_id)))[-12:]}" for event_id in order_items_df['id_purchase']]

# order_items_df 컬럼 순서 정리
columns = ['id', 'order_id', 'created_from_event_id', 'user_id', 'product_id', 'general', 'timesale', 'created_at', 'status', 'shipped_at', 'delivered_at', 'returned_at']
order_items_df = order_items_df[columns]

In [ ]:
# 4


# promotion_id 컬럼 생성 및 설정 

# 데이터 로드 
users_df = pd.read_csv("./data/users.csv", usecols=['id', 'birth', 'gender'],parse_dates=['birth'])
promotions_df = pd.read_csv("./data/promotions.csv", parse_dates=['start_date', 'end_date'])

# 컬럼 표준화 
promotions_df = promotions_df.rename(columns={'id': 'promo_id'})
users_df = users_df.rename(columns={'id': 'user_id'})

# users 생일 월 컬럼 추가
users_df['birth_month'] = users_df['birth'].dt.month

# order_items 주문연도, 주문월 컬럼 추가 
order_items_df['order_year'] = order_items_df['created_at'].dt.year
order_items_df['order_month'] = order_items_df['created_at'].dt.month

# order_items 테이블에 유저 생일 월 추가 (left Join)
order_items_df = pd.merge(order_items_df, users_df, on='user_id', how='left')

# 1) 정기 이벤트 프로모션 기간 매칭

# 주문 시점이 정기 프로모션 기간에 해당하면 event_promo_id 부여 

# promotions_df에서 정기(설/추석, 여름/가을 블랙 프라이데이) 프로모션 id 추출
regular_promos = promotions_df.loc[promotions_df['promotion_type'] == '정기'].copy()

# 정렬 
regular_promos = regular_promos.sort_values('start_date')

# order_items_df 정렬(created_at 기준)
order_items_df = order_items_df.sort_values('created_at')


# start_date 기준 주문 시점이 정기 프로모션 기간에 해당하면 promotion_id 부여
order_items_df = pd.merge_asof(order_items_df, regular_promos[['promo_id', 'name', 'start_date', 'end_date']], 
                               left_on='created_at', right_on='start_date', direction='backward')

# end_date 범위 내에 있는 주문만 promotion_id 유지
range_in_end = (order_items_df['end_date'].notna()) & (order_items_df['created_at'] < order_items_df['end_date'] + timedelta(days=1))
order_items_df['promotion_id'] = np.where(range_in_end, order_items_df['promo_id'], None)
# 2) 생일 프로모션: 유저의 생일 달 첫 구매 (+타임세일 제외)

# 타임세일 제외 
general_purchase = order_items_df.loc[order_items_df['timesale'] == False]

# 유저의 생일 달 == 구매달 
is_birth_month = (general_purchase['order_month'] == general_purchase['birth_month'])

# 유저별+연도별 생일 달 첫 주문 ID 구하기
birth_first_order_ids = (
    general_purchase[is_birth_month]
    .sort_values(["user_id", "order_year", "created_at"])
    .groupby(["user_id", "order_year"]) ['order_id']
    .first()
)

# 그 주문 ID에 해당하는 모든 행들의 인덱스 뽑기
birth_first_order_index = order_items_df.loc[order_items_df['order_id'].isin(birth_first_order_ids)].index

order_items_df.loc[birth_first_order_index, 'promotion_id'] = promotions_df.loc[promotions_df['name'] == '생일자 프로모션', 'promo_id'].values[0]
order_items_df.loc[birth_first_order_index, 'name'] = '생일자 프로모션'


# 3) 타임 세일 프로모션 
order_items_df.loc[order_items_df['timesale'] == True, 'promotion_id'] = promotions_df.loc[promotions_df['name'] == '타임세일', 'promo_id'].values[0]
order_items_df.loc[order_items_df['timesale'] == True, 'name'] = '타임세일'

columns = ['id', 'order_id', 'user_id', 'product_id', 'general', 'timesale', 'created_at', 'status', 'shipped_at', 'delivered_at', 'returned_at', 'promotion_id',  'created_from_event_id']
order_items_df = order_items_df[columns]

In [ ]:
# num_of_items 설정 함수 생성

# ---------------------------
# 1. 카테고리 그룹 매핑
#    - 세부 sub_category를 몇 가지 대표 그룹으로 묶음
#    - 그룹별로 구매 수량 분포를 정의하여 관리 용이성 확보
# ---------------------------
CATEGORY_GROUPS = {
    "outer": [
        '블루종', '레더 재킷', '코트', '패딩', '카디건', '기타 아우터'
    ],
    "shoes_bags": [
        '스니커즈', '운동화', '부츠', '로퍼', '샌들', '슬리퍼', '구두',
        '백팩', '크로스백', '숄더백', '기타 가방'
    ],
    "tops_bottoms": [
        '반소매 티셔츠', '긴소매 티셔츠', '맨투맨', '후드 티셔츠',
        '셔츠/블라우스', '니트', '데님 팬츠', '슬랙스', '조거 팬츠',
        '숏 팬츠', '기타 바지'
    ],
    "onepiece_skirt": [
        '미니 원피스', '셔츠 원피스', '미디 원피스', '니트 원피스', '맥시 원피스',
        '미니 스커트', '롱 스커트', '미디 스커트'
    ],
    "underwear": [
        '브라', '팬티', '이너웨어', '세트 속옷', '속바지'
    ],
    "accessory": [
        '모자', '벨트', '시계', '양말', '장갑', '안경/선글라스', '기타 악세사리'
    ]
}

# ---------------------------
# 2. 분포 정의 (DISTRIBUTIONS)
#    - 각 그룹/세부 속성별로 구매 수량 분포 정의
#    - 예: 팬티는 여러 장 구매하는 경향 반영, 아우터는 대부분 1개
# ---------------------------
DISTRIBUTIONS = {
    "outer":        {1: 0.97, 2: 0.03},                                # 아우터: 사실상 1개
    "shoes_bags":   {1: 0.95, 2: 0.05},                                # 신발/가방도 거의 1개
    "tops_bottoms": {1: 0.82, 2: 0.15, 3: 0.03},                       # 기본 의류: 2~3개 가능
    "onepiece_skirt": {1: 0.94, 2: 0.06},                              # 원피스/스커트: 1개 중심
    "underwear_bra":   {1: 0.88, 2: 0.12},                             # 브라: 보통 1~2개
    "underwear_panty": {1: 0.40, 2: 0.35, 3: 0.15, 4: 0.07, 5: 0.03},  # 팬티: 3~5개까지 현실적
    "underwear_other": {1: 0.65, 2: 0.25, 3: 0.07, 4: 0.03},           # 속바지/이너웨어: 2~3개 가능
    "accessory":    {1: 0.96, 2: 0.03, 3: 0.01}                        # 액세서리: 묶음 가능성 소량 반영
}

# ---------------------------
# 3. sub_category → 그룹명 매핑 딕셔너리
#    - 빠른 검색을 위해 모든 sub_category를 그룹명으로 매핑
# ---------------------------
subcat_to_group = {}
for group, subs in CATEGORY_GROUPS.items():
    for s in subs:
        subcat_to_group[s] = group

# ---------------------------
# 4. 프로모션 승수 정의
#    - 프로모션 타입별로 "2개 이상" 구매 확률을 증가시킴
#    - 일반: 그대로, 정기전: 1.25배, 타임세일: 1.40배
# ---------------------------
PROMO_MULTIPLIER = {
    "일반": 1.00,
    "정기": 1.25,   # 정기전
    "상시": 1.40,   # 타임세일
}

def apply_promo_boost(dist: dict, promo_type: str) -> np.ndarray:
    """
    주어진 구매 수량 분포(dist)에 프로모션 승수를 적용
    - 프로모션이 있는 경우, 수량 ≥ 2인 항목의 확률을 승수만큼 증가
    - 확률 전체를 다시 정규화
    
    input:
        dist (dict): {수량: 확률}
        promo_type (str): 프로모션 타입 ('일반','정기','상시')
    
    return:
        qty (np.ndarray): 가능한 수량
        prob (np.ndarray): 해당 수량의 확률 벡터
    """
    items = sorted(dist.items())  # (qty, prob)
    qty = np.array([k for k,_ in items], dtype=int)
    prob = np.array([v for _,v in items], dtype=float)

    mult = PROMO_MULTIPLIER.get(str(promo_type).lower(), 1.0)
    if mult != 1.0:
        prob[qty >= 2] *= mult
        prob /= prob.sum()
    
    return qty, prob

# ---------------------------
# 5. 샘플링 함수
# ---------------------------
rng = np.random.default_rng(42)

def get_base_dist(sub_category: str):
    """
    주어진 sub_category에 대해 기본 구매 수량 분포(dict)를 반환합니다.
    
    - 언더웨어(브라/팬티/기타)는 세부적으로 분리하여 각기 다른 분포를 사용
    - 그 외의 경우, sub_category → 그룹명 매핑(subcat_to_group)에 따라 그룹 단위 분포를 사용
    - 매핑이 없으면 기본값으로 'accessory' 그룹 분포를 사용
    
    input:
        sub_category : 상품의 세부 카테고리 (예: '팬티', '코트', '반소매 티셔츠' 등)(str)
        
    
    return:
        dict: 구매 수량 분포 {수량: 확률} (예: {1: 0.85, 2: 0.15} → 1개 구매 확률 85%, 2개 구매 확률 15%)
    """
    group = subcat_to_group.get(sub_category, "accessory")
    if group == "underwear":
        if sub_category == "브라":
            return DISTRIBUTIONS["underwear_bra"]
        elif sub_category == "팬티":
            return DISTRIBUTIONS["underwear_panty"]
        else:
            return DISTRIBUTIONS["underwear_other"]
    return DISTRIBUTIONS.get(group, DISTRIBUTIONS["accessory"])

def sample_num_of_items_vectorized(sub_categories: pd.Series, promo_types: pd.Series) -> np.ndarray:
    """
    대규모 데이터셋에 대해 벡터화 방식으로 num_of_items 생성
    
    input:
        sub_categories: 상품 sub_category (Series)
        promo_types   : 프로모션 타입 (Series) → '일반','정기','상시'
    
    return:
        np.ndarray: 각 row별 샘플링된 구매 수량
    """
    out = np.empty(len(sub_categories), dtype=int)

    # 조합별로 한번만 분포 계산 → 해당 인덱스에 일괄 샘플
    keys = pd.DataFrame({"sub_category": sub_categories.values,
                         "promotion_type": promo_types.fillna("일반").values})
    
    for (sc, pt), idx in keys.groupby(["sub_category","promotion_type"]).groups.items():
        base = get_base_dist(sc)
        qty, prob = apply_promo_boost(base, pt)
        n = len(idx)
        u = rng.random(n)
        cum = prob.cumsum()
        out[idx] = qty[np.searchsorted(cum, u)] # 누적 확률 분포에서 랜덤 샘플링
    return out

In [ ]:
# 5
# 정기 프로모션 기간 중 정기 프로모션 최종 판별 및 num_of_items 생성

# 데이터 로드 
products_df = pd.read_csv("./data/products.csv", usecols=['id', 'sub_category', 'retail_price'])
products_df = products_df.rename(columns={'id': 'product_id'})

# order_items_df + product_df (left join) 
order_items_df = pd.merge(order_items_df, products_df, left_on='product_id', right_on='product_id', how='left')

# ----------------
# num_of_items 컬럼 생성
order_items_df = pd.merge(order_items_df, promotions_df, left_on='promotion_id', right_on='promo_id', how='left')
order_items_df = order_items_df.drop(columns=['promo_id'])

order_items_df["num_of_items"] = sample_num_of_items_vectorized(
    order_items_df["sub_category"],
    order_items_df["promotion_type"]
)
# ----------------

# 정기 프로모션 행 추출 
regular_promo_df = order_items_df.loc[order_items_df['promotion_type'] == '정기'].copy()

regular_promo_df['total_price'] = regular_promo_df['retail_price'] * regular_promo_df['num_of_items']

regular_promo_sum_df = regular_promo_df.groupby(['promotion_id','order_id'], as_index=False)['total_price'].sum()

regular_promo_sum_df = pd.merge(regular_promo_sum_df, promotions_df, left_on='promotion_id', right_on='promo_id', how='left')

higher_mask = regular_promo_sum_df['total_price'] >= regular_promo_sum_df['minimum_sale_price']

higher_order_ids = regular_promo_sum_df.loc[higher_mask, 'order_id'].unique().tolist()
lower_order_ids  = regular_promo_sum_df.loc[~higher_mask, 'order_id'].unique().tolist()

# 프로모션 기간 중 구매자가 프로모션에 대한 무지로 인한 프로모션 미적용
# λ 설정 (예: 평균 2% 정도 프로모션 놓침)
num_miss = np.random.poisson(0.02 * len(higher_order_ids))  # 전체 개수 대비 비율 반영
num_miss = random.sample(higher_order_ids, num_miss)in(num_miss, len(higher_order_ids))  # 안전장치: 리스트보다 많지 않게
missed_ids = random.sample(higher_order_ids, num_miss)

order_items_df.loc[(order_items_df['order_id'].isin(missed_ids)) & (order_items_df['promotion_type'] == '정기'), 'promotion_id'] = None
order_items_df.loc[(order_items_df['order_id'].isin(lower_order_ids)) & (order_items_df['promotion_type'] == '정기'), 'promotion_id'] = None

# 컬럼 정리 및 order_items_df로 저장 (prmotion_id가 새로 조정되었기 때문에 다시 promotions랑 조인해야함.)
columns = ['id', 'order_id', 'user_id', 'product_id', 'sub_category', 'promotion_id', 'status', 'created_at', 'shipped_at', 'delivered_at', 'returned_at', 'retail_price', 'num_of_items', 'created_from_event_id']
order_items_df = order_items_df[columns]

In [131]:
order_items_df.head()

,id,order_id,user_id,product_id,sub_category,promotion_id,status,created_at,shipped_at,delivered_at,returned_at,retail_price,num_of_items
0,oi-243f3f83-b1c,or-7949a6c1242d,44091,6277ead6,슬리퍼,1,Purchased,2022-01-01 00:01:24,2022-01-01 21:44:32.738778,2022-01-03 00:21:22.563303,NaT,34600,1
1,oi-6850d5b8-3f7,or-d25190faeced,54098,9a48e33a,기타 가방,14,Purchased,2022-01-01 00:02:07,2022-01-01 21:45:15.738778,2022-01-03 00:22:05.563303,NaT,43100,1
2,oi-3cf5e834-10f,or-1dbf72891640,80078,62f1995c,니트,1,Purchased,2022-01-01 00:02:17,2022-01-01 21:45:25.738778,2022-01-03 00:22:15.563303,NaT,87100,2
3,oi-58404489-33a,or-69bb2c4b6472,4993,d4c7b0df,이너웨어,1,Purchased,2022-01-01 00:02:18,2022-01-01 21:45:26.738778,2022-01-03 00:22:16.563303,NaT,20800,1
4,oi-41e9a902-06e,or-bbc191a03533,93217,665b3f4b,긴소매 티셔츠,1,Purchased,2022-01-01 00:02:32,2022-01-01 21:45:40.738778,2022-01-03 00:22:30.563303,NaT,40400,2


In [ ]:
# 6
# retail_price, sale_price 컬럼 생성 및 설정 

# order_items_df + promotions_df left join (prmotion_id가 새로 조정되었기 때문에 다시 promotions랑 조인해야함.)
order_items_df = pd.merge(order_items_df, promotions_df, left_on='promotion_id', right_on='promo_id', how='left')

# 1) sale_price(각 주문행의 최종 가격)
order_items_df['sale_price'] = order_items_df['num_of_items'] * order_items_df['retail_price']

# 2) 타임세일 sale_price 설정
timesale_df = order_items_df.loc[order_items_df['name'] == '타임세일'].copy()
# sale_price 계산
timesale_df['sale_price'] = timesale_df['retail_price'] * timesale_df['num_of_items'] * (1 - timesale_df['discount_rate'])
order_items_df.loc[timesale_df.index, 'sale_price'] = timesale_df['sale_price']

# 3) 생일자, 설/추석, 가을/여름 블랙프라이데이 sale_price 설정
# 생일자, 설/추석, 가을/여름 블랙프라이데이 행 추출 
birth_regular_df = order_items_df.loc[(order_items_df['promotion_type'] == '정기') | (order_items_df['name'] == '생일자 프로모션')].copy()

# Total_price 컬럼 생성
birth_regular_df['total_price'] = birth_regular_df['retail_price'] * birth_regular_df['num_of_items']

# 할인 금액 컬럼 생성
birth_regular_df['discount_price'] = birth_regular_df['total_price'] * birth_regular_df['discount_rate']

# 정렬
birth_regular_df = birth_regular_df.sort_values(by=['user_id', 'order_id', 'retail_price'], ascending=[True, True, False])

# 누적 할인 금액 컬럼 생성
birth_regular_df['cum_discount_price'] = birth_regular_df.groupby(['user_id', 'order_id'])['discount_price'].cumsum()

birth_regular_df['prev_order_id'] = birth_regular_df.groupby('user_id')['order_id'].shift(1)

# 누적 할인 금액이 최대 할인 금액 보다 높은 인덱스, 낮은 인덱스 추출 
over_idx = birth_regular_df.loc[birth_regular_df['cum_discount_price'] > birth_regular_df['maximum_discount_price']].index.tolist()
under_idx = birth_regular_df.loc[birth_regular_df['cum_discount_price'] <= birth_regular_df['maximum_discount_price']].index.tolist()
# 인덱스에 맞는 sale_price 설정 
if under_idx:
    order_items_df.loc[under_idx, 'sale_price'] = birth_regular_df['total_price'] - birth_regular_df['discount_price']
if over_idx:
    for idx, row in birth_regular_df.loc[over_idx].iterrows():
        if row['prev_order_id'].isna():
            order_items_df.at[idx, 'sale_price'] = birth_regular_df['total_price'] - birth_regular_df['discount_price'] 
            + (birth_regular_df['cum_discount_price'] - birth_regular_df['maximum_discount_price'])
        else:
            order_items_df.at[idx, 'sale_price'] = birth_regular_df['total_price']

# 최종 order_items_df 생성 
columns = ['id', 'order_id', 'user_id', 'product_id', 'status', 'created_at','shipped_at', 'delivered_at', 'returned_at','promotion_id', 'name', 'discount_rate', 'retail_price', 'num_of_items', 'sale_price', 'created_from_event_id']
order_items_df = order_items_df[columns]
order_items_df = order_items_df.rename(columns={'name': 'promotion_name'})

# promotion_id 컬럼 int형으로 변환 (+ None -> -1)
order_items_df['promotion_id'] = order_items_df['promotion_id'].fillna(-1).astype('int64')

# created_at, shipped_at, delivered_at, returned_at 시간 처리 
order_items_df['created_at'] = order_items_df['created_at'].dt.strftime('%Y-%m-%d %H:%M:%S')
order_items_df['created_at'] = pd.to_datetime(order_items_df['created_at'])

order_items_df['shipped_at'] = order_items_df['shipped_at'].dt.strftime('%Y-%m-%d %H:%M:%S')
order_items_df['shipped_at'] = pd.to_datetime(order_items_df['shipped_at'])

order_items_df['delivered_at'] = order_items_df['delivered_at'].dt.strftime('%Y-%m-%d %H:%M:%S')
order_items_df['delivered_at'] = pd.to_datetime(order_items_df['delivered_at'])

order_items_df['returned_at'] = order_items_df['returned_at'].dt.strftime('%Y-%m-%d %H:%M:%S')
order_items_df['returned_at'] = pd.to_datetime(order_items_df['returned_at'])

In [135]:
# order_items_df 저장
order_items_df.to_csv("./data/order_items.csv", index=False)

## 연간 구매액 기준 회원 등급 부여 (users.csv)

In [146]:
def assign_user_tiers(users: pd.DataFrame, order_items: pd.DataFrame) -> pd.DataFrame:
    """
    연간 구매액(Annual Spend)을 기준으로 사용자별 회원 등급(loyal, regular, dormant)을 부여합니다.

    로직:
        1) order_items 테이블에서 user_id 기준으로 연간 구매액(total_spend)을 집계
        2) users 테이블과 병합하여 각 사용자의 연간 구매액을 계산
        3) 구매액 구간을 기준으로 tier 컬럼을 생성
           - loyal   : 상위 고객 (연간 구매액 >= 500,000원)
           - regular : 일반 고객 (연간 구매액 100,000원 ~ 500,000원 미만)
           - dormant : 저활성 고객 (연간 구매액 < 100,000원 또는 구매 이력 없음)

    input:
        users (pd.DataFrame): 사용자 정보 테이블. 최소한 'id' 컬럼 필요.
        order_items (pd.DataFrame): 주문 상세 테이블. 최소한 'user_id', 'sale_price' 컬럼 필요.

    return:
        pd.DataFrame: tier 컬럼이 추가된 users DataFrame
    """
    # 1) 연간 구매액 집계
    annual_spend = (
        order_items.groupby("user_id")["sale_price"]
        .sum()
        .reset_index()
        .rename(columns={"sale_price": "annual_spend"})
    )

    # 2) users 테이블과 병합
    users = users.merge(annual_spend, left_on="id", right_on="user_id", how="left")
    users["annual_spend"] = users["annual_spend"].fillna(0)

    # 3) 구간 기준 tier 부여
    def classify_tier(spend):
        if spend >= 500_000:
            return "loyal"
        elif spend >= 100_000:
            return "regular"
        else:
            return "dormant"

    users["user_type"] = users["annual_spend"].apply(classify_tier)

    return users

In [ ]:
users_df = pd.read_csv('./data/users.csv', parse_dates=['birth', 'created_at'])

In [ ]:
users_df = assign_user_tiers(users_df, order_items_df)
column_order = ['id', 'created_at', 'user_type', 'first_name', 'last_name', 'email',
                'age', 'birth', 'gender', 'address', 'traffic_source']
users_df = users_df[column_order]

In [151]:
users_df.to_csv('./data/users.csv', index=False)

# Orders

In [ ]:
# 데이터 로드 
order_items_df = pd.read_csv("./data/order_items.csv", parse_dates=['created_at','shipped_at', 'delivered_at', 'returned_at'])
users_df = pd.read_csv("./data/users.csv", usecols=['id', 'gender'])
users_df = users_df.rename(columns={'id': 'user_id'})

# 주문 상태 정리 함수
def map_order_status(group):
    statuses = set(group)
    if statuses == {'Purchased'}:
        return 'Complete'
    elif statuses == {'Returned'}:
        return 'Complete'
    elif statuses == {'Purchased', 'Returned'}:
        return 'Completed'
    elif 'Cancelled' in statuses:
        return 'Cancelled'
    else:
        return 'Processing'


        
# orders 테이블 생성 + gender
orders_df = (
    order_items_df.groupby('order_id').agg(
        user_id=('user_id', 'first'),
        status=('status', map_order_status),
        created_at=('created_at', 'max'),
        shipped_at=('shipped_at', 'max'),
        delivered_at=('delivered_at', 'max'),
        returned_at=('returned_at', 'max'),
        num_of_items=('num_of_items', 'sum')
    )
    .reset_index()
)

# gender컬럼 생성
orders_df = pd.merge(orders_df, users_df, on='user_id', how='left')

# 컬럼 정리
columns = ['order_id', 'user_id', 'status', 'gender', 'created_at', 'shipped_at', 'delivered_at', 'returned_at', 'num_of_items']
orders_df = orders_df.rename(columns={'order_id': 'id'})
orders_df = orders_df[columns]

In [138]:
# orders_df 저장
orders_df.to_csv("./data/orders.csv", index=False)

```